In [1]:
import pandas as pd
import os
import math
import json
import numpy as np

import unidecode
from collections import Counter
import re
from datetime import datetime

import warnings

In [2]:
warnings.filterwarnings("ignore")

## Utilities

### Validation

In [3]:
def validate_num_tables_in_sheet(true_value, expected_value):
    if expected_value!=true_value:
        error = json.dumps({
            "message": f"Error: Inconsistency in `data_map.json`. The expected number of tables must be \
                '{expected_value}' but data is available for only {true_value} tables"
        })
        raise AssertionError(error)
    
def validate_sheet_size(sheet_name, true_value, expected_value=1):
    if not (true_value>=expected_value):
        error = json.dumps({
            "message": f"Error: Sheet '{sheet_name}' has no data"
        })
        raise AssertionError(error)

In [4]:
def validate_and_resolve_columns_inconsistencies(sheet_name, true_columns, expected_columns):
    new_true_columns = true_columns.copy()
    new_true_columns = [columns_inconsistencies[col] if col in columns_inconsistencies else col for col in true_columns]
    mismatch_columns = set(expected_columns).difference(set(new_true_columns))
    if len(mismatch_columns)>0:
        return [], []
    return new_true_columns

def validate_table_columns(sheet_name, true_value, expected_value):
    true_columns = pre_process_columns(true_value)
    expected_columns = pre_process_columns(expected_value)
    true_columns = validate_and_resolve_columns_inconsistencies(
        sheet_name, true_columns, expected_columns)
    # Checking if expected columns match
    if len(true_columns)==0 and len(expected_columns)==0 and len(set(expected_columns).difference(set(true_columns)))!=0:
        error=json.dumps({
            "message": f"The columns in sheet '{sheet_name}' are not matched. Expected columns must be '{expected_columns_new}'"
        })
        raise AssertionError(error)
        
    return true_columns, expected_columns

In [5]:
def spanish_to_english(value):
    value = "".join(value.lower().split(" "))
    # Decode string to nearest unicode
    value = unidecode.unidecode(value)
    return value

def validate_student_name_consistency_helper(sheet_name, true_names, apparent_names):
    """
    Verifies the student names in the both lists. In case of mismatch in the `apparent_names`, through an assertion error with row number and name
    
    @input
    sheet_name -> Name of the sheet
    true_names -> List of ground truth names (may be duplicate but not always)
    apparent_names -> List of apparent names (may contain duplicates)
    
    @output
    True if matches, else False
    """
    
    error = {
        "sheet_name": f"{sheet_name}",
        "row_number": 0
    }
    
    # Test each name in aparent_name
    found_mismatch = False
    for i in range(len(apparent_names)):
        error["row_number"] = i
        apparent_name = apparent_names[i]
        if apparent_name not in true_names and apparent_name!="Demo":
            # Try more advanced methods
            found = False
            for true_name in true_names:
                if spanish_to_english(true_name)==spanish_to_english(apparent_name):
                    found = True
                    break
            
            if not found:
                error["message"] = f"Student name {apparent_name} not found"
                found_mismatch = True
            
    return not found_mismatch, error

def validate_student_name_consistency_driver(dataframe_dict): 
    """
    Validates the student names in the whole sheet
    """
    
    # Test for student name consistency in a company's data
    ## Test Student data in `Skill Building`
    true_names = dfs["students"].fullname.to_list()
    apparent_names = dfs["skill_building_line"].participant.to_list()
    status, _ = validate_student_name_consistency_helper("Skill Building", true_names, apparent_names)
    if not status:
        raise AssertionError(_)

    ## Test Student data in `Final Exercise`
    apparent_names = dfs["final_exercise_line"].participant.to_list()
    status, _ = validate_student_name_consistency_helper("Final Exercise", true_names, apparent_names)
    if not status:
        raise AssertionError(_)

    ## Test Student data in `Instructor Comments`
    apparent_names = dfs["instructor_comments"].participant.to_list()
    status, _ = validate_student_name_consistency_helper("Instructor Comments", true_names, apparent_names)
    if not status:
        raise AssertionError(_)

### Helpers

In [6]:
def get_all_directories():
    return [f"{data_path}/{name}" for name in os.listdir(f"{data_path}") if os.path.isdir(f"{data_path}/{name}")]

def get_all_files_in_directory(dir_name, file_extension):
    """
    Get all the files in a given company directory
    
    @input
    dir_name: Name of company directory
    file_extension: The file extension to read
    
    @output
    List of files in a directory
    """
    if os.path.exists(dir_name):
        files = os.listdir(dir_name)
        allowed_files = list()
        for file in files:
            if file.split(".")[-1] == file_extension:
                allowed_files.append(f"{dir_name}/{file}")
        return allowed_files
    else:
        error = json.dumps({
            "message": f"'{dir_name}' does not exist",
        })
        raise AssertionError(error)

def get_all_files_in_all_directories(file_extension):
    """
    Real all the companies data in the root directory
    
    @Input
    file_extension: The file extension to read
    
    @output
    List of all excel files
    """
    all_files = dict()
    for dir_name in get_all_directories():
        data = get_all_files_in_directory(dir_name, file_extension)
        all_files[dir_name] = data
    all_excel_files = [file for key in all_files for file in all_files[key] if file.split(".")[-1] == file_extension]
    
    return all_excel_files

In [7]:
def pre_process_columns(columns):
    return [_.replace(r"#", "").strip().lower() for _ in columns]

def parse_sheet_with_single_dataframes(sheet_name, sheet_data, column_names):
    """
    Parse a sheet which has only one dataframe (ONLY)
    
    @inputs
    sheet_name -> Name of the sheet
    sheet_data -> Sheet data in pd.Dataframe
    meta_data -> A list of columns
    """
    assert sheet_data is not None and type(sheet_data)==pd.DataFrame and sheet_data.shape[0]!=0, f"Error: Sheet '{sheet_name}' is not correct"
    assert set(column_names).difference(set(sheet_data.columns))!=0, f"Error: The columns in sheet '{sheet_name}' are not matched. \
        Expected columns must be '{column_names}'"
    df = sheet.loc[:, column_names]
    more_than_half = math.floor(len(df.columns)/2)
    df = df[df.isna().sum(axis=1)/(more_than_half) < 1 ]
    return df

In [8]:
def parse_excel_sheet(excel_file_path, excel_data_map, str_columns):
    """
    Parse the given excel sheet.
    Note: All `student` is replaced with `participant`
    
    @input
    excel_file_path -> File path for the excel file
    excel_data_map -> Ground truth map for excel file
    str_columns -> Columns names which must be converted to str
    
    @output
    Returns all the tables in the excel file
    """
    
    if not os.path.exists(excel_file_path) and excel_file_path.split(".")[-1]!='xlsx':
        error = json.dumps({
            "message": "Excel file doesn't exist or incorrect file extension"
        })
        raise AssertionError(error)
        
    # Read excel file
    excel_data = pd.ExcelFile(excel_file_path)
    
    print("****Verifying the excel file****")
    # Test for sheet names
    expected_sheet_names = [_["sheet_name"] for _ in excel_data_map]
    actual_sheets_names = excel_data.sheet_names
    if len(set(actual_sheets_names).intersection(set(expected_sheet_names)))!=len(expected_sheet_names):
        error = json.dumps({
            "message": f"Sheet names does not match. Current allowed values are: {expected_sheet_names}"
        })
        raise AssertionError(error)
        
    full_excel_dict = dict()
    for _ in excel_data_map:
        sheet_name = _["sheet_name"]
        header = _["header"]
        num_tables = _["num_table"]
        validate_num_tables_in_sheet(true_value=len(_["meta_data"]), expected_value=num_tables)

        # Do not read the sheet if it is not required
        if "required" in _ and not _["required"]:
            continue
        for i in range(num_tables):
            # Do not read the sheet if it is not required
            if "required" in _ and not _["meta_data"][i]["required"]:
                continue
            table_name = _["meta_data"][i]["table_name"]
            non_null_col = None
            if "non_null_col" in _["meta_data"][i]:
                non_null_col = _["meta_data"][i]["non_null_col"]
            
            sheet = pd.read_excel(excel_data, sheet_name, header=header)
            validate_sheet_size(sheet_name, sheet.shape[0], 1)

            true_columns = list(sheet.columns)
            expected_columns = list(_["meta_data"][i]["columns"].keys())
            true_columns, expected_columns = validate_table_columns(sheet_name, true_columns, expected_columns)
            
            # Read the table
            sheet.columns = true_columns
            df = sheet.loc[:, expected_columns]
            df.columns = expected_columns
            # Put NaN for empty rows
            for col in df.columns:
                df[col] = df[col].apply(lambda x: np.NaN if len(str(x).strip())==0 else x)
            
            # pre-process certain columns
            for col in df.columns:
                if col in str_columns:
                    df[col] = df[col].apply(lambda x: x.strip() if type(x)==str else x)
            
            # Remove rows with null data for non null column
            df = df.dropna(subset=[non_null_col])
            
            print(f"=> Sheet '{sheet_name}',\n table: '{table_name}' verified and read. \
                \nThe columns are: {expected_columns}. \nTotal rows: {df.shape[0]}")
            full_excel_dict[table_name] = df
            
    print("****Verified and read the excel file's metadata successfully****")
    
    return full_excel_dict

In [9]:
def parse_csv_sheet(company_directory, final_exercise_vehicle_ids):
    """
    Pase the given csv sheet. The sheet represents a final exercise. 
    It links the final_exercise_line with csv by a factor of 3
    
    @input
    company_directory -> Company's directory for all the csv file path
    final_exercise_vehicle_ids -> All the vehicle ids in the final_exercise_line dataframe
    
    @output
    Returns all the csv data with format:
    {
        vehicle_id: {
            "exercise": pd.DataFrame,
            "exercise_agg": pd.DataFrame,
        }
    }
    """
    
    csv_file_paths = get_all_files_in_directory(company_directory, "csv")
    if len(csv_file_paths)==0:
        error = {
            "message": "There is no csv file in the company directory"
        }
        raise AssertionError(error)
    
    exercise_data = dict()
    student_counts_for_vehicle_dict = dict(Counter(final_exercise_vehicle_ids))
    print(f"****Total student counts in a vehicle: {student_counts_for_vehicle_dict}****")
    for csv_file_path in csv_file_paths:
        vehicle_id = int(csv_file_path.split(".")[-2][-1])
        print(f"****Reading data for vehicle id {vehicle_id}****")
        if vehicle_id not in student_counts_for_vehicle_dict:
            error = {
                "message": f"File {csv_file_path} does not have vehicle id in it's name. \
                    Expected v1 or v2 or v3 .. as suffix in file name"
            }
            raise AssertionError(error)
        num_students = student_counts_for_vehicle_dict[vehicle_id]
        expected_rows = num_students*3
        with open(csv_file_path, "r") as file:
            exercise_date = file.readline()
            exercise_date = re.findall(r"[0-9]{2}/[0-9]{2}/[0-9]{4} [0-9]{2}:[0-9]{2}", exercise_date)[0]

        df = pd.read_csv(csv_file_path, skiprows=1)
        # Add test date columns
        df["Test Date"] = exercise_date

        # Remove last 4 rows
        df_exercise = df.iloc[:-4, :]
        df_exercise_agg = df.iloc[-4:, :]

        # Remove rows with Time = 0
        # df_exercise = df_exercise[df_exercise['Time (s)']!=0]
        
        # Test if exercise have correct number of results
        difference = df_exercise.shape[0] - expected_rows
        print("Total row difference in csv: ", difference)
        if difference>0:
            error = {
                "message": f"Data could not be registered. There are {difference} number of extra rows in sheet \
                    {csv_file_path}"
            }
            raise AssertionError(error)
        elif difference<0:
            error = {
                "message": f"Data could not be registered. There are {-difference} rows missing in sheet {csv_file_path}"
            }
            raise AssertionError(error)
            
        exercise_data[vehicle_id] = {
            "exercise": df_exercise,
            "exercise_agg": df_exercise_agg
        }
    
    return exercise_data

## Main code

In [10]:
data_path = "./../../../static/database_migration_files"
data_map_path = "./../static_data/data_map.json"
columns_inconsistencies_path = "./../static_data/columns_inconsistencies.json"
allowed_extensions = ["xlsx", "csv"]

In [11]:
# Reading true data
with open(data_map_path) as file:
    data_map = json.load(file)
excel_data_map = data_map["excel"]["sheets"]

with open(columns_inconsistencies_path) as file:
    columns_inconsistencies = json.load(file)

excel_str_columns = ["location", "program", "client", "country", "units", "make", "exercise", "name", "last name", 
               "company", "fullname", "gender", "participant", "comment"]

In [12]:
## Not for Julio 18 - GALEAM: Less rows in csv
## Junio 30 Julio 1 - BASF: Wrong car number (5th does not exist)

In [13]:
# Get all company directories
all_directories = get_all_directories()

In [14]:
all_directories

['./../../../static/database_migration_files/Junio 30 Julio 1 - BASF',
 './../../../static/database_migration_files/2022 - March 17-18 - Willow Springs - OE',
 './../../../static/database_migration_files/2021 - Willow Springs - Aug 3 - gdba',
 './../../../static/database_migration_files/Julio 18 - GALEAM',
 './../../../static/database_migration_files/2022 - Jul 14 - LVMS - AUS',
 './../../../static/database_migration_files/Junio 24 y 25']

In [15]:
company_dir = all_directories[0]
print("Parsing data for company dir: ", company_dir)
# Get all excel files in a directory
excel_files = get_all_files_in_directory(company_dir, "xlsx")
dfs = parse_excel_sheet(excel_files[0], excel_data_map, str_columns=excel_str_columns)
    
# Validate student names
validate_student_name_consistency_driver(dfs)
print("****Student names verified****")

# Parse csv files in a directory
vehicle_ids = dfs["final_exercise_line"].car.astype(int).to_list()
df_exercise_details = parse_csv_sheet(company_dir, vehicle_ids)

Parsing data for company dir:  ./../../../static/database_migration_files/Junio 30 Julio 1 - BASF
****Verifying the excel file****
=> Sheet 'Course Values',
 table: 'course_generals' verified and read.                 
The columns are: ['date', 'location', 'program', 'client', 'country', 'units']. 
Total rows: 1
=> Sheet 'Course Values',
 table: 'car_information' verified and read.                 
The columns are: ['car', 'make', 'latacc']. 
Total rows: 4
=> Sheet 'Course Values',
 table: 'skill_building' verified and read.                 
The columns are: ['exercise', 'chord', 'mo']. 
Total rows: 2
=> Sheet 'Course Values',
 table: 'final_exercise' verified and read.                 
The columns are: ['ideal_time sec', 'cone penalty sec', 'door penalty sec']. 
Total rows: 4
=> Sheet 'Students',
 table: 'students' verified and read.                 
The columns are: ['name', 'last name', 'company', 'gender', 'birthday', 'fullname']. 
Total rows: 9
=> Sheet 'Skill Building',
 table: '

### Data pre-processing

In [16]:
def raise_preprocessing_error(message, row, method):
    error = {
        "message": message,
        "row": row,
        "method": method
    }
    raise AssertionError(error)

#### Table `students`

In [17]:
# dfs['students']['birthday'].dt.strftime('%Y%m%d')
def make_student_id(row):
    unique_id = "{}{}{}{}".format(
        row['last name'][0].upper(),
        row['last name'][-1].upper(),
        row['name'][0].upper(),
        row['birthday'].strftime('%Y%m%d')
    )
    return unique_id

# Convert string to datetime
dfs['students']['birthday'] = pd.to_datetime(dfs['students']['birthday'])
dfs['students']['studentId'] = dfs['students'].apply(lambda row: make_student_id(row), axis=1)

In [18]:
student_name_id_dict = dfs['students'].set_index("fullname")["studentId"].to_dict()

#### Assign unique_id to all sheets

In [19]:
student_name_id_dict = dfs['students'].set_index("fullname")["studentId"].to_dict()

In [20]:
dfs['final_exercise_line']["studentId"] = dfs['final_exercise_line'].participant.map(student_name_id_dict)
dfs['skill_building_line']["studentId"] = dfs['skill_building_line'].participant.map(student_name_id_dict)
dfs['instructor_comments']["studentId"] = dfs['instructor_comments'].participant.map(student_name_id_dict)

#### Table `skill_building`

In [21]:
dfs['skill_building'] = dfs['skill_building'].set_index("exercise")

In [22]:
def calculate_radius(row):
    try:
        x = row['chord']
        y = row['mo']
        radius = (x**2) / (8*y) + (y/2)
        return radius
    except Exception as e:
        raise_preprocessing_error(message=str(e), row=row.name+1, method="Calculting Radius")

In [23]:
dfs['skill_building']['radius'] = dfs['skill_building'].apply(lambda row: calculate_radius(row), axis=1)

#### Table `skill_building_line`

In [24]:
dfs['skill_building']

,chord,mo,radius
exercise,,,
Slalom,50.0,3.625,88.019397
Lane Change,100.0,13.640,98.462229


In [25]:
dfs['skill_building_line']['car'] = dfs['skill_building_line']['car'].astype(int)
dfs['skill_building_line']['cones'] = dfs['skill_building_line']['cones'].fillna(0)

In [26]:
car_lat_acc_map = dfs["car_information"].set_index("car")["latacc"].to_dict()

In [27]:
def ex_percentage(row):
    try:
        x = row[['v1', 'v2', 'v3']].mean()
        sd = row[['v1', 'v2', 'v3']].std()
        cns = row['cones']
        y = round(x, 2)
        z = row['speed req']
        p = ((y / z))
        res = 0
        if sd < 3 and cns == 0:
            res = round(p, 2)
        return res
    except Exception as e:
        raise_preprocessing_error(str(e), row.name+1, "Calculating ex percentage")
        
def v_percentage(row, car_lat_acc_map):
    """
    Calculate %_of_vehicles. 
    Note: Depends on car_lat_acc_map, and dfs['skill_building']['radius']
    """
    try:
        vx = row[['v1', 'v2', 'v3']].mean()
        sd = row[['v1', 'v2', 'v3']].std()
        ex = row['exercise']
        R = dfs['skill_building'].loc[ex.strip(), 'radius']
        v = round(vx, 2)
        LA = ((v**2) / (R*15))
        res = round(LA, 2)/car_lat_acc_map[row["car"]]
        if res < 1:
            pass
        else:
            res = 0
        if sd < 3:
            pass
        else:
            res = 0
        return res
    except Exception as e:
        raise_preprocessing_error(str(e), row.name+1, "Calculating v percentage")

In [28]:
dfs.keys()

dict_keys(['course_generals', 'car_information', 'skill_building', 'final_exercise', 'students', 'skill_building_line', 'final_exercise_line', 'instructor_comments'])

In [29]:
# Units of Measure (Metric or Imperial)
units = dfs['course_generals'].loc[0, 'units']
kms_per_mile = 1.609344

# Conversion formula 1.609344 kms per mile
if units == 'MPH':
    pass
else:
    dfs['skill_building_line']['speed req'] = round((dfs['skill_building_line']['speed req']/kms_per_mile), 0).astype(int)
    dfs['skill_building_line']['v1'] = round((dfs['skill_building_line']['v1']/kms_per_mile), 0).astype(int)    
    dfs['skill_building_line']['v2'] = round((dfs['skill_building_line']['v2']/kms_per_mile), 0).astype(int)    
    dfs['skill_building_line']['v3'] = round((dfs['skill_building_line']['v3']/kms_per_mile), 0).astype(int)    

In [30]:
dfs['skill_building_line']['%_of_exercise'] = dfs['skill_building_line'].apply(lambda row: ex_percentage(row), axis=1).astype('float')
dfs['skill_building_line']['%_of_vehicle'] = dfs['skill_building_line'].apply(lambda row: v_percentage(row, car_lat_acc_map), axis=1).astype('float')

#### Table `final_exercise`

In [31]:
ideal_time = pd.to_timedelta(dfs['final_exercise'].loc[0, 'ideal_time sec'], unit='s')
c_penalty = pd.to_timedelta(dfs['final_exercise'].loc[0, 'cone penalty sec'], unit='s')
g_penalty = pd.to_timedelta(dfs['final_exercise'].loc[0, 'door penalty sec'], unit='s')

#### Table `DataFinalExerciseComputed`

In [32]:
df_passed_ex = dfs['skill_building_line'][['participant','exercise','cones','%_of_vehicle']]

In [33]:
df_passed_ex = df_passed_ex.query("cones==0 and `%_of_vehicle` >= .1").reset_index(drop=True)

In [34]:
df_passed_ex = df_passed_ex.groupby(
    by = ['participant', 'exercise']
).agg(
    start_score = ('%_of_vehicle', "min"),
    end_score = ('%_of_vehicle', "max")
).unstack(level=1)

In [35]:
df_passed_ex_enhance = dfs['skill_building_line'].replace(0, np.nan).groupby(
    by=['participant', 'exercise']
).agg(
    count=("exercise", "size"),
    passed=("%_of_exercise", "count"),
    av_score=("%_of_exercise", "mean"),
    start_score=("%_of_vehicle", "min"),
    end_score=("%_of_vehicle", "max"),
).replace(np.nan, 0).unstack(level=1)

In [36]:
df_passed_ex_enhance['LnCh Passed'] = df_passed_ex_enhance['passed']['Lane Change']/df_passed_ex_enhance["count"]["Lane Change"]
df_passed_ex_enhance['Slalom Passed'] = df_passed_ex_enhance['passed']['Slalom']/df_passed_ex_enhance["count"]["Slalom"]

In [37]:
# Slalom average
df_vehicle_pc_avg = dfs['skill_building_line'].replace(np.nan, 0).groupby(
    by=['participant', 'exercise']
).agg(
    vehicle_pc_avg=('%_of_vehicle','mean')
).unstack(level=1)

In [38]:
# Renaming columns
df_final_exercise = dfs['final_exercise_line'].copy()
if "pressure" in df_final_exercise.columns:
    df_final_exercise.rename(columns={"pressure": "stress", "doors": "gates", "time":"g_time"}, inplace=True)
# Changing types
df_final_exercise["car"] = df_final_exercise["car"].astype(int)
df_final_exercise[['stress', 'cones', 'gates']] = df_final_exercise[['stress', 'cones', 'gates']].fillna(0).astype(int)

#Time conversion to Time Delta
df_final_exercise['g_time'] = pd.to_timedelta(df_final_exercise['g_time'], unit='s')

In [39]:
# Final exercise - final time calculations
def final_result_calculations(row):
    """
    Uses data from `final_exercise`. 
    Note: Not used
    """
    cns = row['cones']
    gts = row['gates']
    tme1 = row['g_time']
    g_tme = (tme1 + (cns * c_penalty) + (gts * g_penalty))
    f_time = (-(((g_tme / ideal_time)*100)-200)/100)
    f_time = round(f_time, 2)
    return f_time

In [40]:
df_final_exercise['final_result'] = df_final_exercise.apply(lambda row: final_result_calculations(row), axis=1)

In [41]:
df_final_exercise["f_time"] = df_final_exercise.apply(
    lambda row: row['g_time'] + ((row['cones']*c_penalty) + (row['gates']*g_penalty)), axis=1)

#### Vehicle csv's processing

In [42]:
def process_cars_csv_data(df_exercise_details):
    csv_car_results = list()
    for car in df_exercise_details:
        c_df = df_exercise_details[car]['exercise'].rename(
            columns={
                'UTC time (At Start)':'UTC_Time',
                'Time (s)' : 'rev_slalom'
            }
        )
        c_df['car'] = car
        c_time = c_df[['car', 'rev_slalom', 'UTC_Time']][::3].reset_index(drop=True)

        c_slalom = c_df[['Speed (Avg) (mph)', 'Speed (Std Dev) (mph)']][1::3].reset_index(drop=True)
        c_slalom.rename(columns={
            'Speed (Avg) (mph)': 'slalom_avg', 
            'Speed (Std Dev) (mph)':'slalom_std_dev', 
        }, inplace=True)

        c_LnCh = c_df[['Speed (Avg) (mph)', 'Speed (Std Dev) (mph)']][2::3].reset_index(drop=True)
        c_LnCh.rename(columns={
            'Speed (Avg) (mph)': 'LnCh_avg', 
            'Speed (Std Dev) (mph)':'LnCh_std_dev', 
        }, inplace=True)

        # Frame variables
        c_frames = [c_time, c_slalom, c_LnCh]

        # Final DataFrames
        c_result = pd.concat(c_frames, join='inner', axis=1)

        # Creating Time Deltas
        c_result['rev_slalom'] = c_result['rev_slalom'].astype(float)
        c_result['rev_slalom'] = pd.to_timedelta(c_result['rev_slalom'], unit='s')

        # Converting time to datetime64
        c_result['UTC_Time'] = (pd.to_datetime(c_result['UTC_Time'], format=('%H:%M:%S.%f')).dt.time)

        # Changind types to float
        float_type_cols = ['slalom_avg', 'slalom_std_dev', 'LnCh_avg', 'LnCh_std_dev']
        c_result[float_type_cols] = c_result[float_type_cols].astype(float)
        c_result.rename(columns={'car' : 'car_ID'}, inplace=True)
        c_result['car_ID'] = c_result['car_ID'].astype(int)

        csv_car_results.append(c_result)
        
    # Creating a sigle DataFrame
    df_vbox = pd.concat(csv_car_results, ignore_index=True).sort_values(by=['UTC_Time']).reset_index().drop(['index'], axis=1)
    return df_vbox

In [43]:
# MSE slalom and LnCh percentages
def mse_slalom_pc(row):
    vx = row['slalom_avg']
    ex = 'Slalom'
    R = dfs['skill_building'].loc[ex.strip(), 'radius']
    v = round(vx, 2)
    LA = ((v**2) / (R*15))
    if LA >= car_lat_acc_map[row["car_ID"]]:
        res = 0
    else:
        res = round(LA, 2) / car_lat_acc_map[row["car_ID"]]
    return int((round(res, 2))*100)

def mse_LnCh_pc(row):
    vx = row['LnCh_avg']
    ex = 'Lane Change'
    R = dfs['skill_building'].loc[ex.strip(), 'radius']
    v = round(vx, 2)
    LA = ((v**2) / (R*15))

    if LA >= car_lat_acc_map[row["car_ID"]]:
        res = 0
    else:
        res = round(LA, 2) / car_lat_acc_map[row["car_ID"]]

    return int((round(res, 2))*100)

In [44]:
df_vbox = process_cars_csv_data(df_exercise_details)

In [45]:
df_vbox['slalom'] = df_vbox.apply(lambda row: mse_slalom_pc(row), axis=1)
df_vbox['LnCh'] = df_vbox.apply(lambda row: mse_LnCh_pc(row), axis=1)

#### Continue final exercise

In [46]:
# Assign df_vbox variables to df_final_exercise
df_final_exercise['rev_slalom'] = df_vbox['rev_slalom']
df_final_exercise['slalom'] = df_vbox['slalom']
df_final_exercise['LnCh'] = df_vbox['LnCh']

In [47]:
# Remove demo rows
# demo_rows = df_final_exercise[df_final_exercise['participant'] == 'Demo'].index
# df_final_exercise.drop(demo_rows, inplace=True)
df_final_exercise = df_final_exercise[df_final_exercise['participant'] != 'Demo']

In [48]:
df_final_exercise.head()

,id,exercise,participant,stress,car,cones,gates,g_time,studentId,final_result,f_time,rev_slalom,slalom,LnCh
3,4,Final Exercise,Miguel Angel Ortiz Herrera,0,1,1,0,0 days 00:01:10.330000,OAM19770929,0.95,0 days 00:01:12.330000,0 days 00:00:14.300000,71,74
4,5,Final Exercise,Luis Alberto Zozaya Ordoñez,0,1,0,0,0 days 00:01:16,ZZL19930105,0.90,0 days 00:01:16,0 days 00:00:15.810000,61,54
5,6,Final Exercise,Héctor Ruiz Flores,0,1,1,0,0 days 00:01:22.030000,RSH19681019,0.78,0 days 00:01:24.030000,0 days 00:00:19.450000,52,43
6,7,Final Exercise,Jesús Cruz González,0,1,4,0,0 days 00:01:21.420000,CZJ19641114,0.70,0 days 00:01:29.420000,0 days 00:00:18.090000,70,60
7,8,Final Exercise,Juan Guerrero Ortiz,0,1,4,0,0 days 00:01:13.390000,GZJ19570731,0.82,0 days 00:01:21.390000,0 days 00:00:15.730000,72,54


In [49]:
df_final_exercise['rev_%'] = df_final_exercise.apply(lambda x: round((x['rev_slalom'] / x['g_time']), 2), axis=1)

In [50]:
final_required_cols = ['exercise', 'studentId', 'participant', 'car', 'stress', 'rev_slalom', 'rev_%', 'slalom', 'LnCh', 'cones', 'gates',
       'f_time', 'final_result']
# df_final_exercise = 
df_final_exercise = df_final_exercise[final_required_cols].reset_index(drop=True)

## Data Models

In [52]:
dfs.keys()

dict_keys(['course_generals', 'car_information', 'skill_building', 'final_exercise', 'students', 'skill_building_line', 'final_exercise_line', 'instructor_comments'])

In [53]:
# Countries
to_send = dfs["course_generals"][["country", "units"]].to_dict(orient="records")
to_send

[{'country': 'MX', 'units': 'KPH'}]

In [54]:
# Students
df_students = dfs["students"].rename(columns={
    "name": "firstName",
    "last name": "lastName",
}).drop(columns=["fullname"])
df_students.birthday = df_students.birthday.dt.date

In [55]:
df_students

,firstName,lastName,company,gender,birthday,studentId
0,Juan,Guerrero Ortiz,Basf,M,1957-07-31,GZJ19570731
1,German Enoc,Velazco Gallardo,Basf,M,1977-03-07,VOG19770307
2,Miguel Angel,Ortiz Herrera,Basf,M,1977-09-29,OAM19770929
3,Jesús,Cruz González,Basf,M,1964-11-14,CZJ19641114
4,Luis Alberto,Zozaya Ordoñez,Basf,M,1993-01-05,ZZL19930105
5,Fernando Alejandro,Rojas Barrios,Basf,M,1973-11-08,RSF19731108
6,Luis Alejandro,Novelo Ramírez,Basf,M,1962-12-26,NZL19621226
7,Luis Martín,Lima González,Basf,M,1963-02-10,LZL19630210
8,Héctor,Ruiz Flores,Polioles,M,1968-10-19,RSH19681019


In [56]:
to_send = df_students.to_dict(orient="records")
to_send

[{'firstName': 'Juan',
  'lastName': 'Guerrero Ortiz',
  'company': 'Basf',
  'gender': 'M',
  'birthday': datetime.date(1957, 7, 31),
  'studentId': 'GZJ19570731'},
 {'firstName': 'German Enoc',
  'lastName': 'Velazco Gallardo',
  'company': 'Basf',
  'gender': 'M',
  'birthday': datetime.date(1977, 3, 7),
  'studentId': 'VOG19770307'},
 {'firstName': 'Miguel Angel',
  'lastName': 'Ortiz Herrera',
  'company': 'Basf',
  'gender': 'M',
  'birthday': datetime.date(1977, 9, 29),
  'studentId': 'OAM19770929'},
 {'firstName': 'Jesús',
  'lastName': 'Cruz González',
  'company': 'Basf',
  'gender': 'M',
  'birthday': datetime.date(1964, 11, 14),
  'studentId': 'CZJ19641114'},
 {'firstName': 'Luis Alberto',
  'lastName': 'Zozaya Ordoñez',
  'company': 'Basf',
  'gender': 'M',
  'birthday': datetime.date(1993, 1, 5),
  'studentId': 'ZZL19930105'},
 {'firstName': 'Fernando Alejandro',
  'lastName': 'Rojas Barrios',
  'company': 'Basf',
  'gender': 'M',
  'birthday': datetime.date(1973, 11, 8),

In [57]:
# Programs
to_send = dfs["course_generals"][["program"]].rename(
    columns={"program": "name"}
).to_dict(orient="records")
print(to_send)

[{'name': 'Habilidades de Manejo Avanzadas [Anti-Secuestro]'}]


In [58]:
# Venues
to_send = to_send = dfs["course_generals"][["location", "country"]].rename(
    columns={"location": "name"}
).to_dict(orient="records")
print(to_send)

[{'name': 'OFF ROAD Training Center', 'country': 'MX'}]


In [59]:
# Course
df_course = dfs["course_generals"][["location", "program", "date"]].rename(
    columns={"location": "venue", "date": "eventDate"},
)
df_course["idealTime"] = int(dfs["final_exercise"].loc[0, "ideal_time sec"])
df_course["conePenalty"] = int(dfs["final_exercise"].loc[0, "cone penalty sec"])
df_course["gatePenalty"] = int(dfs["final_exercise"].loc[0, "door penalty sec"])
to_send = df_course.to_dict(orient="records")
print(to_send)

[{'venue': 'OFF ROAD Training Center', 'program': 'Habilidades de Manejo Avanzadas [Anti-Secuestro]', 'eventDate': Timestamp('2022-06-30 00:00:00'), 'idealTime': 69, 'conePenalty': 2, 'gatePenalty': 5}]


In [60]:
# CoursesStudents
# Create a course, student mapping after creation

In [61]:
# Vehicles
df_vehicles = dfs["car_information"][["make", "latacc"]]
df_vehicles["name"] = df_vehicles["make"]
df_vehicles = df_vehicles.drop_duplicates()
to_send = df_vehicles.to_dict(orient="records")
print(to_send)

[{'make': 'VW Jetta 2.5', 'latacc': 0.93, 'name': 'VW Jetta 2.5'}]


In [62]:
# Comments
to_send = dfs["instructor_comments"][["studentId", "comment"]].to_dict(orient="records")
print(to_send)

[{'studentId': 'GZJ19570731', 'comment': 'Juan tuvo un desempeño correcto, y ha mostrado una clara evolución respecto a años anteriores. quizás aún tiene que trabajar en mejorar la concentración para lograr mejor precisión. Va por buen camino.'}, {'studentId': 'VOG19770307', 'comment': 'Enoc siempre se ha mostrado callado, sin el afán de demostrarle nada a nadie, pero cumpliendo satisfactoriamente todo lo que se le pide. Aún no está en su nivel óptimo, pero él mismo lo sabe y se esfuerza para cada vez llegar más lejos.'}, {'studentId': 'OAM19770929', 'comment': 'Miguel se mostró impaciente por probar nuevos retos, siempre queriendo llegar más lejos. Su nivel de habilidades es notablemente bueno, por momentos entra en competencia externa y desvía su enfoque, y es esta la razón de los pocos errores que cometió.'}, {'studentId': 'CZJ19641114', 'comment': 'Jesus aún tiene problemas para trabajar bajo presión, así como para conducción en reversa. Tiene las bases, sin embargo, tiene poca con

In [63]:
# ExercisesSelected
to_send = dfs['skill_building'].reset_index()[["exercise", "chord", "mo"]].to_dict(orient="records")
print(to_send)

[{'exercise': 'Slalom', 'chord': 50.0, 'mo': 3.625}, {'exercise': 'Lane Change', 'chord': 100.0, 'mo': 13.64}]


In [64]:
# DataExercises
to_send = dfs['skill_building_line'][["studentId", "exercise", "car", "speed req", "v1", "v2", "v3", "%_of_exercise", "%_of_vehicle"]].rename(
    columns={"%_of_vehicle": "pVehicle",
            "%_of_exercise": "pExercise",
            "speed req": "speedReq",
            "car": "vehicle"}
).assign(penalties=np.NaN).to_dict(orient="records")
# to_send

## Student report

In [65]:
df_final_exercise

,exercise,studentId,participant,car,stress,rev_slalom,rev_%,slalom,LnCh,cones,gates,f_time,final_result
0,Final Exercise,OAM19770929,Miguel Angel Ortiz Herrera,1,0,0 days 00:00:14.300000,0.20,71,74,1,0,0 days 00:01:12.330000,0.95
1,Final Exercise,ZZL19930105,Luis Alberto Zozaya Ordoñez,1,0,0 days 00:00:15.810000,0.21,61,54,0,0,0 days 00:01:16,0.90
2,Final Exercise,RSH19681019,Héctor Ruiz Flores,1,0,0 days 00:00:19.450000,0.24,52,43,1,0,0 days 00:01:24.030000,0.78
3,Final Exercise,CZJ19641114,Jesús Cruz González,1,0,0 days 00:00:18.090000,0.22,70,60,4,0,0 days 00:01:29.420000,0.70
4,Final Exercise,GZJ19570731,Juan Guerrero Ortiz,1,0,0 days 00:00:15.730000,0.21,72,54,4,0,0 days 00:01:21.390000,0.82
5,Final Exercise,VOG19770307,German Enoc Velazco Gallardo,1,0,0 days 00:00:17.950000,0.23,74,82,3,0,0 days 00:01:25.170000,0.77
6,Final Exercise,LZL19630210,Luis Martín Lima González,1,0,0 days 00:00:22.190000,0.25,48,59,3,1,0 days 00:01:40.140000,0.55
7,Final Exercise,RSF19731108,Fernando Alejandro Rojas Barrios,1,0,0 days 00:00:17.570000,0.23,73,46,0,0,0 days 00:01:15.670000,0.90
8,Final Exercise,NZL19621226,Luis Alejandro Novelo Ramírez,1,0,0 days 00:00:15.900000,0.21,65,57,1,0,0 days 00:01:17.340000,0.88
9,Final Exercise,LZL19630210,Luis Martín Lima González,1,1,0 days 00:00:17.400000,0.22,62,66,3,1,0 days 00:01:28.810000,0.71


- df_passed_ex_enhance -> counts_df  
- dfs['skill_building_line'] -> course_df
- dfs['students'] -> students_df
- df_vehicle_pc_avg -> slalom_avg
- df_passed_ex -> passed_ex_df

In [66]:
# Group Variables
gasnor = int((df_passed_ex_enhance['count']['Slalom'].agg('mean'))) # group_average_slalom_runs
gaspoce = int((df_passed_ex_enhance['Slalom Passed'].mean())*100) # group_average_slalom_prcnt_completed
gasaoep = int(((df_passed_ex_enhance['av_score']['Slalom'].mean())*100)) # group_average_slalom_ex_prcnt
gasaovc = int((dfs['skill_building_line'].loc[dfs['skill_building_line']['exercise'] == 'Slalom']['%_of_vehicle'].mean())*100) # group_average_slalom_vehicle_control
galnor = int((df_passed_ex_enhance['count']['Lane Change'].agg('mean'))) #.astype(int) # group_average_lnch_runs
galpoce = int((df_passed_ex_enhance['LnCh Passed'].mean())*100) # group_average_lnch_prcnt_completed
galaoep = int(((df_passed_ex_enhance['av_score']['Lane Change'].mean())*100)) # group_average_lnch_ex_prcnt
galaovc = int((dfs['skill_building_line'].loc[dfs['skill_building_line']['exercise'] == 'Lane Change']['%_of_vehicle'].mean())*100) # group_average_lnch_vehicle_control
mseg_t_pre = str(df_final_exercise['f_time'].mean())
mseg_t = mseg_t_pre[mseg_t_pre.find(':')+1 : mseg_t_pre.find('.')+3 : ] # mse_group_av_time
mseg_c = int((df_final_exercise['cones'].mean())) # mse_group_av_cones
mseg_g = int((df_final_exercise['gates'].mean())) # mse_group_av_gates
mseg_perf = int((df_final_exercise['final_result'].mean())*100) # mse_group_av_performance
mseg_per = int((df_final_exercise['final_result'].quantile())*100) # mse_group_av_percentile
mse_obj = str(pd.to_timedelta(dfs['final_exercise'].loc[0, 'ideal_time sec'], unit='s'))[10:] # mse_objective_obj
mseg_rev_pre = str(df_final_exercise['rev_slalom'].mean()) # strings mse_group_used_in_rev
"""New Change in slicing"""
mseg_rev = re.findall('days 00:00:(.*\..{2}|.*$)', mseg_rev_pre)[0] # strings mse_group_used_in_rev <- Chnaged May 2021
mseg_rev_pc = int(round(((df_final_exercise['rev_%'].mean())*100), 0))

'16.38'

In [97]:
df_final_exercise['rev_slalom'].mean()

Timedelta('0 days 00:00:16.379166666')

In [96]:
type(df_final_exercise['rev_slalom'].mean())

pandas._libs.tslibs.timedeltas.Timedelta

In [94]:
mseg_rev_pre

'0 days 00:00:16.379166666'

In [93]:
mseg_rev

'16.37'

In [67]:
# Temporary Variables waiting for code
msed_per= np.nan # MSE Driver Percentile (Quantile)
msed_rev = np.nan # MSE Driver % Used in Reverse
msed_rev_time = np.nan # MSE Driver Reverse Time
paragraph = np.nan # Lead Instructor Feedback

In [68]:
country = dfs['course_generals'].loc[0, 'country']
make = dfs['car_information'].loc[0, 'make']
program = dfs['course_generals'].loc[0, 'program']
course_date = dfs['course_generals'].loc[0, 'date']
latacc = dfs['car_information'].loc[0, 'latacc']

#Final report DataFrame Lineup
df_report = pd.DataFrame(dfs['students'], columns=['fullname', 'studentId'])
df_report['company'] = np.nan
df_report['program'] = program
df_report['date'] = course_date

# Versiones en ambos idiomas
if country == 'MX':
    df_report['vehicle'] = (make + ' (Capacidad ' + latacc.astype(str) + 'g)')
else:
    df_report['vehicle'] = (make + ' (' + latacc.astype(str) + 'g Capability)')

In [69]:
df_passed_ex_enhance

count             passed         \
exercise                         Lane Change Slalom Lane Change Slalom   
participant                                                              
Fernando Alejandro Rojas Barrios           7      7           6      6   
German Enoc Velazco Gallardo              10      8           7      6   
Héctor Ruiz Flores                         7      8           4      7   
Jesús Cruz González                        7      8           5      7   
Juan Guerrero Ortiz                        6      9           3      6   
Luis Alberto Zozaya Ordoñez                7      8           5      8   
Luis Alejandro Novelo Ramírez              7      8           5      7   
Luis Martín Lima González                  7      9           7      5   
Miguel Angel Ortiz Herrera                 5      5           5      2   

                                    av_score           start_score            \
exercise                         Lane Change    Slalom Lane Change    Slalom   
participant                                                                    
Fernando Alejandro Rojas Barrios    0.988333  1.020000    0.688172  0.698925   
German Enoc Velazco Gallardo        0.982857  1.023333    0.763441  0.720430   
Héctor Ruiz Flores                  0.977500  0.957143    0.741935  0.440860   
Jesús Cruz González                 0.966000  0.975714    0.741935  0.548387   
Juan Guerrero Ortiz                 1.030000  0.973333    0.806452  0.698925   
Luis Alberto Zozaya Ordoñez         1.032000  1.011250    0.741935  0.623656   
Luis Alejandro Novelo Ramírez       0.994000  1.022857    0.774194  0.752688   
Luis Martín Lima González           0.988571  1.014000    0.763441  0.655914   
Miguel Angel Ortiz Herrera          1.034000  1.050000    0.774194  0.666667   

                                   end_score           LnCh Passed  \
exercise                         Lane Change    Slalom               
participant                                                          
Fernando Alejandro Rojas Barrios    0.838710  0.881720    0.857143   
German Enoc Velazco Gallardo        0.838710  0.881720    0.700000   
Héctor Ruiz Flores                  0.892473  0.795699    0.571429   
Jesús Cruz González                 0.806452  0.838710    0.714286   
Juan Guerrero Ortiz                 0.892473  0.784946    0.500000   
Luis Alberto Zozaya Ordoñez         0.946237  0.870968    0.714286   
Luis Alejandro Novelo Ramírez       0.892473  0.870968    0.714286   
Luis Martín Lima González           0.860215  0.838710    1.000000   
Miguel Angel Ortiz Herrera          0.924731  0.903226    1.000000   

                                 Slalom Passed  
exercise                                        
participant                                     
Fernando Alejandro Rojas Barrios      0.857143  
German Enoc Velazco Gallardo          0.750000  
Héctor Ruiz Flores                    0.875000  
Jesús Cruz González                   0.875000  
Juan Guerrero Ortiz                   0.666667  
Luis Alberto Zozaya Ordoñez           1.000000  
Luis Alejandro Novelo Ramírez         0.875000  
Luis Martín Lima González             0.555556  
Miguel Angel Ortiz Herrera            0.400000

In [70]:
# Slalom variables
df_report['s_no_runs'] = pd.merge(left=df_report, right=(df_passed_ex_enhance['count']['Slalom']).astype(int), left_on='fullname', right_index=True)['Slalom']
df_report['s_passed'] = pd.merge(left=df_report, right=df_passed_ex_enhance['passed']['Slalom'], left_on='fullname', right_index=True)['Slalom']
df_report['prcnt_s_pass'] = pd.merge(left=df_report, right=round((df_passed_ex_enhance['Slalom Passed'])*100, 0), left_on='fullname', right_index=True)['Slalom Passed']
df_report['avg_ex_control_s'] = pd.merge(left=df_report, right=round((df_passed_ex_enhance['av_score']['Slalom'])*100, 0), left_on='fullname', right_index=True)['Slalom']
df_report['avg_v_control_s'] = pd.merge(left=df_report, right=round((df_vehicle_pc_avg['vehicle_pc_avg']['Slalom'])*100, 0), left_on='fullname', right_index=True)['Slalom']
df_report['slalom_max'] = pd.merge(left=df_report, right=round((df_passed_ex['end_score']['Slalom'])*100), left_on='fullname', right_index=True)['Slalom']

#LnCh Variables
df_report['lc_no_runs'] = pd.merge(left=df_report, right=(df_passed_ex_enhance['count']['Lane Change']).astype(int), left_on='fullname', right_index=True)['Lane Change']
df_report['lc_passed'] = pd.merge(left=df_report, right=df_passed_ex_enhance['passed']['Lane Change'], left_on='fullname', right_index=True)['Lane Change']
df_report['prcnt_lc_pass'] = pd.merge(left=df_report, right=round((df_passed_ex_enhance['LnCh Passed'])*100, 0), left_on='fullname', right_index=True)['LnCh Passed']
df_report['avg_ex_control_lc'] = pd.merge(left=df_report, right=round((df_passed_ex_enhance['av_score']['Lane Change'])*100, 0), left_on='fullname', right_index=True)['Lane Change']
df_report['avg_v_control_lc'] = pd.merge(left=df_report, right=round((df_vehicle_pc_avg['vehicle_pc_avg']['Lane Change'])*100, 0), left_on='fullname', right_index=True)['Lane Change']
df_report['lnch_max'] = pd.merge(left=df_report, right=round((df_passed_ex['end_score']['Lane Change'])*100), left_on='fullname', right_index=True)['Lane Change']

In [71]:
# Setup the right company
def set_right_company_student_report(row, student_company_map, client_name):
    """
    Set's the company for a student in the student report. If the company is not known for a student then put client name
    """
    company = client_name
    if row["studentId"] in student_company_map:
        company = student_company_map[row["studentId"]]
    return company


student_company_map = dfs['students'].set_index("studentId")["company"].to_dict()
client = dfs['course_generals'].loc[0, 'client']
df_report["company"] = df_report.apply(lambda row: set_right_company_student_report(row, student_company_map, client), axis=1)

In [72]:
# Including the right comments
student_comment_map = dfs["instructor_comments"].set_index("studentId")["comment"].to_dict()
df_report["comments"] = df_report["studentId"].map(student_comment_map)

In [73]:
df_final_exercise.loc[df_final_exercise['final_result'] < .8]

,exercise,studentId,participant,car,stress,rev_slalom,rev_%,slalom,LnCh,cones,gates,f_time,final_result
2,Final Exercise,RSH19681019,Héctor Ruiz Flores,1,0,0 days 00:00:19.450000,0.24,52,43,1,0,0 days 00:01:24.030000,0.78
3,Final Exercise,CZJ19641114,Jesús Cruz González,1,0,0 days 00:00:18.090000,0.22,70,60,4,0,0 days 00:01:29.420000,0.70
5,Final Exercise,VOG19770307,German Enoc Velazco Gallardo,1,0,0 days 00:00:17.950000,0.23,74,82,3,0,0 days 00:01:25.170000,0.77
6,Final Exercise,LZL19630210,Luis Martín Lima González,1,0,0 days 00:00:22.190000,0.25,48,59,3,1,0 days 00:01:40.140000,0.55
9,Final Exercise,LZL19630210,Luis Martín Lima González,1,1,0 days 00:00:17.400000,0.22,62,66,3,1,0 days 00:01:28.810000,0.71


In [74]:
# Important change in how dates are read (Cambio Importante en como se leen las fechas)
# Final Exercise Variables - Possible setting as table for multiple occurances
mse_report = pd.DataFrame(df_final_exercise.replace(np.nan, '-')
                          .drop(['exercise', 'rev_slalom'], axis=1)
                        )

mse_report['f_time'] = mse_report['f_time'].astype(str).str.extract('days 00:(.*\..{2}|.*$)')

if country == 'MX':
    mse_report['stress'].replace((1, 0), ('Alto', 'Bajo'), inplace=True)
else:
    mse_report['stress'].replace((1, 0), ('High', 'Low'), inplace=True)

mse_report[['rev_%', 'final_result']] = mse_report[['rev_%', 'final_result']].apply(lambda x: x*100).astype(int)
mse_report.rename(columns={'rev_%' : 'rev_pc'}, inplace=True)

for i, r in mse_report.iterrows():
    if '.' in r.f_time:
        pass
    else:
        r = r.copy()
        ov = r.f_time
        ov = ov+".01"
        mse_report.loc[i, 'f_time'] = ov

mse_report = mse_report.groupby('participant').apply(lambda x: x.to_dict(orient='records'))

### Jinga variable reads

In [75]:
jinga_dir = "../../../static/reports"
tmplt_data = pd.read_excel(f'{jinga_dir}/jinja_variables.xlsx', skiprows=2)
tmplt_context = dict(zip(tmplt_data['var'], tmplt_data['value']))

In [81]:
df_final_exercise

,exercise,studentId,participant,car,stress,rev_slalom,rev_%,slalom,LnCh,cones,gates,f_time,final_result
0,Final Exercise,OAM19770929,Miguel Angel Ortiz Herrera,1,0,0 days 00:00:14.300000,0.20,71,74,1,0,0 days 00:01:12.330000,0.95
1,Final Exercise,ZZL19930105,Luis Alberto Zozaya Ordoñez,1,0,0 days 00:00:15.810000,0.21,61,54,0,0,0 days 00:01:16,0.90
2,Final Exercise,RSH19681019,Héctor Ruiz Flores,1,0,0 days 00:00:19.450000,0.24,52,43,1,0,0 days 00:01:24.030000,0.78
3,Final Exercise,CZJ19641114,Jesús Cruz González,1,0,0 days 00:00:18.090000,0.22,70,60,4,0,0 days 00:01:29.420000,0.70
4,Final Exercise,GZJ19570731,Juan Guerrero Ortiz,1,0,0 days 00:00:15.730000,0.21,72,54,4,0,0 days 00:01:21.390000,0.82
5,Final Exercise,VOG19770307,German Enoc Velazco Gallardo,1,0,0 days 00:00:17.950000,0.23,74,82,3,0,0 days 00:01:25.170000,0.77
6,Final Exercise,LZL19630210,Luis Martín Lima González,1,0,0 days 00:00:22.190000,0.25,48,59,3,1,0 days 00:01:40.140000,0.55
7,Final Exercise,RSF19731108,Fernando Alejandro Rojas Barrios,1,0,0 days 00:00:17.570000,0.23,73,46,0,0,0 days 00:01:15.670000,0.90
8,Final Exercise,NZL19621226,Luis Alejandro Novelo Ramírez,1,0,0 days 00:00:15.900000,0.21,65,57,1,0,0 days 00:01:17.340000,0.88
9,Final Exercise,LZL19630210,Luis Martín Lima González,1,1,0 days 00:00:17.400000,0.22,62,66,3,1,0 days 00:01:28.810000,0.71


In [ ]:
def prepare_mse_data(studentId, country):
    mse_student = df_final_exercise[df_final_exercise['studentId'] == studentId].index.tolist()
    mse_data = pd.DataFrame(df_final_exercise.iloc[mse_student], columns=['stress', 'f_time', 'revPercent', 'slalom', 'LnCh', 'cones', 'gates', 'final_result'])
    if country == 'MX':
        mse_data['stress'].replace((1, 0), ('Alto', 'Bajo'), inplace=True)
    else:
        mse_data['stress'].replace((1, 0), ('High', 'Low'), inplace=True)
    mse_data['revPercent'] = mse_data['revPercent'].astype(str) + '%'
    mse_data['slalom'] = mse_data['slalom'].astype(str) + '%'
    mse_data['LnCh'] = mse_data['LnCh'].astype(str) + '%'
    mse_data['final_result'] = mse_data['final_result'].astype(str) + '%'
    return mse_data

In [79]:
'-------------------------------------------------------------'
'Nex Table For Final Exercise'
'-------------------------------------------------------------'

student = "Héctor Ruiz Flores"
studentId = "RSH19681019"
mse_student = df_final_exercise[df_final_exercise['studentId'] == studentId].index.tolist()
# finalx_df_slice = finalx_df.iloc[7]
mse_data = pd.DataFrame(df_final_exercise.iloc[mse_student], columns=['stress', 'f_time', 'rev_%', 'slalom', 'LnCh', 'cones', 'gates', 'final_result'])
mse_data[['rev_%', 'final_result']] = (mse_data[['rev_%', 'final_result']]*100).astype(int)
mse_data['f_time'] = mse_data['f_time'].astype(str).str.extract('days 00:(.*\..{2}|.*$)')
if country == 'MX':
    mse_data['stress'].replace((1, 0), ('Alto', 'Bajo'), inplace=True)
else:
    mse_data['stress'].replace((1, 0), ('High', 'Low'), inplace=True)
mse_data['rev_%'] = mse_data['rev_%'].astype(str) + '%'
mse_data['slalom'] = mse_data['slalom'].astype(str) + '%'
mse_data['LnCh'] = mse_data['LnCh'].astype(str) + '%'
mse_data['final_result'] = mse_data['final_result'].astype(str) + '%'

In [137]:
mse_data

,stress,f_time,rev_%,slalom,LnCh,cones,gates,final_result
2,Bajo,01:24.03,24%,52%,43%,1,0,78%
10,Alto,01:13.43,20%,59%,63%,0,0,94%


### Report

In [86]:
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.styles import ParagraphStyle as PS
from reportlab.platypus import PageBreak, Table, NextPageTemplate, Image, Spacer, Flowable, TableStyle
from reportlab.platypus.paragraph import Paragraph
from reportlab.platypus.doctemplate import PageTemplate, BaseDocTemplate
from reportlab.pdfbase.pdfmetrics import registerFont, registerFontFamily
from reportlab.platypus.tableofcontents import TableOfContents
from reportlab.platypus.frames import Frame
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.units import cm, inch
from reportlab.lib import colors, utils
from functools import partial
from reportlab.lib.pagesizes import letter
from PIL import Image as pImage

In [87]:
from docxtpl import DocxTemplate, InlineImage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage, AnnotationBbox)
import jinja2

In [109]:
df_report

,fullname,studentId,company,program,date,vehicle,s_no_runs,s_passed,prcnt_s_pass,avg_ex_control_s,avg_v_control_s,slalom_max,lc_no_runs,lc_passed,prcnt_lc_pass,avg_ex_control_lc,avg_v_control_lc,lnch_max,comments
0,Juan Guerrero Ortiz,GZJ19570731,Basf,Habilidades de Manejo Avanzadas [Anti-Secuestro],2022-06-30,VW Jetta 2.5 (Capacidad 0.93g),9,6,67.0,97.0,58.0,78.0,6,3,50.0,103.0,57.0,86.0,"Juan tuvo un desempeño correcto, y ha mostrado..."
1,German Enoc Velazco Gallardo,VOG19770307,Basf,Habilidades de Manejo Avanzadas [Anti-Secuestro],2022-06-30,VW Jetta 2.5 (Capacidad 0.93g),8,6,75.0,102.0,70.0,88.0,10,7,70.0,98.0,57.0,84.0,"Enoc siempre se ha mostrado callado, sin el af..."
2,Miguel Angel Ortiz Herrera,OAM19770929,Basf,Habilidades de Manejo Avanzadas [Anti-Secuestro],2022-06-30,VW Jetta 2.5 (Capacidad 0.93g),5,2,40.0,105.0,31.0,90.0,5,5,100.0,103.0,86.0,92.0,Miguel se mostró impaciente por probar nuevos ...
3,Jesús Cruz González,CZJ19641114,Basf,Habilidades de Manejo Avanzadas [Anti-Secuestro],2022-06-30,VW Jetta 2.5 (Capacidad 0.93g),8,7,88.0,98.0,64.0,84.0,7,5,71.0,97.0,56.0,81.0,Jesus aún tiene problemas para trabajar bajo p...
4,Luis Alberto Zozaya Ordoñez,ZZL19930105,Basf,Habilidades de Manejo Avanzadas [Anti-Secuestro],2022-06-30,VW Jetta 2.5 (Capacidad 0.93g),8,8,100.0,101.0,78.0,87.0,7,5,71.0,103.0,75.0,95.0,"Luis tiene un gran control sobre el vehículo, ..."
5,Fernando Alejandro Rojas Barrios,RSF19731108,Basf,Habilidades de Manejo Avanzadas [Anti-Secuestro],2022-06-30,VW Jetta 2.5 (Capacidad 0.93g),7,6,86.0,102.0,67.0,88.0,7,6,86.0,99.0,68.0,84.0,"Fer tiene gran actitud ante el aprendizaje, y ..."
6,Luis Alejandro Novelo Ramírez,NZL19621226,Basf,Habilidades de Manejo Avanzadas [Anti-Secuestro],2022-06-30,VW Jetta 2.5 (Capacidad 0.93g),8,7,88.0,102.0,70.0,87.0,7,5,71.0,99.0,71.0,89.0,"Alex ha mejorado mucho a lo largo de los años,..."
7,Luis Martín Lima González,LZL19630210,Basf,Habilidades de Manejo Avanzadas [Anti-Secuestro],2022-06-30,VW Jetta 2.5 (Capacidad 0.93g),9,5,56.0,101.0,59.0,84.0,7,7,100.0,99.0,80.0,86.0,"Lima es un conductor muy cauteloso, por moment..."
8,Héctor Ruiz Flores,RSH19681019,Polioles,Habilidades de Manejo Avanzadas [Anti-Secuestro],2022-06-30,VW Jetta 2.5 (Capacidad 0.93g),8,7,88.0,96.0,62.0,80.0,7,4,57.0,98.0,46.0,89.0,Hector a pesar de contar con menos experiencia...


In [90]:
# Jinja function to assign variables

def create_report(data):
    i = 0
    global student, program, fulldate, vehicle, s_graph, snor, spoce, saoep, saovc, asfpl, l_graph, lnor, lpoce, laoep, laovc, lfpl, msed_rev_time, comment, items, pct_graph, mse_graph, slalom_graph, lnch_graph, rev_graph
    
    performance_report_tmplt = "../../../static/reports/"
    if not os.path.exists("plots") or not os.path.isdir("plots"):
        os.mkdir("plots")
    if not os.path.exists("reports") or not os.path.isdir("reports"):
        os.mkdir("reports")
    
    for index, row in data.iterrows():
        if i > len(data):
            break
        else:
            if country == 'MX':
                doc = DocxTemplate(f'{performance_report_tmplt}/esp_performance_report_tmplt.docx')
            else:
                doc = DocxTemplate(f'{performance_report_tmplt}eng_performance_report_tmplt.docx')
            
            #Define variables for template
            
            if row['fullname'] not in df_final_exercise['participant'].values:
                pass
            else:
                student = row['fullname']
            
            
            
            fullname = student
            studentId = row["studentId"]
            company = row['company']
            program = row['program']
            
            # Date language Format
            if country == 'MX':
                fulldate = row['date'].strftime("%d / %m / %Y")
            else:
                fulldate = row['date'].strftime("%B %d %Y")
            
            vehicle = row['vehicle']
            snor = row['s_no_runs']
            spoce = int(row['prcnt_s_pass'])
            saoep = int(row['avg_ex_control_s'])
            saovc = int(row['avg_v_control_s'])
            sfpl = int(row['slalom_max'])
            lnor = row['lc_no_runs']
            lpoce = int(row['prcnt_lc_pass'])
            laoep = int(row['avg_ex_control_lc'])
            laovc = int(row['avg_v_control_lc']) 
            lfpl = int(row['lnch_max'])
            
            #Graphs Variables
            ax1_slalom_plt = (dfs['skill_building_line'].loc[
                ((dfs['skill_building_line']['studentId'] == studentId) & 
                 (dfs['skill_building_line']['exercise'] == 'Slalom')), '%_of_exercise'
            ].astype(float))*100
            ax1_slalom_plt = ax1_slalom_plt.reset_index(drop=True)
            ax2_slalom_plt = (dfs['skill_building_line'].loc[
                ((dfs['skill_building_line']['studentId'] == studentId) & 
                 (dfs['skill_building_line']['exercise'] == 'Slalom')), '%_of_vehicle'
            ].astype(float))*100
            ax2_slalom_plt = ax2_slalom_plt.reset_index(drop=True)
            
            ax1_LnCh_plt = (dfs['skill_building_line'].loc[
                ((dfs['skill_building_line']['studentId'] == studentId) & 
                 (dfs['skill_building_line']['exercise'] == 'Lane Change')), '%_of_exercise'
            ].astype(float))*100
            ax1_LnCh_plt = ax1_LnCh_plt.reset_index(drop=True)
            ax2_LnCh_plt = (dfs['skill_building_line'].loc[
                ((dfs['skill_building_line']['studentId'] == studentId) & 
                 (dfs['skill_building_line']['exercise'] == 'Lane Change')), '%_of_vehicle'
            ].astype(float))*100
            ax2_LnCh_plt = ax2_LnCh_plt.reset_index(drop=True)
            
            # Instructor comments 
            paragraph = row['comments']
            
            #Slalom Runs Graph
            plt.style.use('seaborn-dark-palette')
            plt.figure(figsize=(6.5,2))
            plt.axhline(y=80, color='#C87867', ls='--', lw=3)
            plt.annotate('Ideal', (0, 90), ha='center', va='center', fontsize=8, color='#C87867')
            plt.axhline(y=100, color='#67BEC8', ls='--', lw=3)
            plt.annotate('Ideal', (0, 110), ha='center', va='center', fontsize=8, color='#67BEC8')
            
            
            # Language Specific
            if country == 'MX':
                plt.plot(ax1_slalom_plt, label = '% Del Ejercicio', linewidth=2, color='#001EBA')
                plt.plot(ax2_slalom_plt, label = '% Del Vehículo', linewidth=2, color='#BA0000')
                plt.title('Resultados de Slalom - ' + student)
            else:
                plt.plot(ax1_slalom_plt, label = '% Of The Exercise', linewidth=2, color='#001EBA')
                plt.plot(ax2_slalom_plt, label = '% Of The Vehicle', linewidth=2, color='#BA0000')
                plt.title(student + ' ' + 'Slalom Results')
            
            plt.ylim(ymin=0, ymax=120)
            plt.legend()
            
            
              #Language specific labels
            if country == 'MX':
                plt.xlabel('Intentos')
            else:
                plt.xlabel('Runs')
            
            plt.ylabel('%')
            plt.tight_layout()
            plt.savefig('plots/Slalom Graph ' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            s_graph = InlineImage(doc, 'plots/Slalom Graph ' + student + '.png')
            
            #Lane Change Graph
            plt.style.use('seaborn-dark-palette')
            plt.figure(figsize=(6.5,2))
            plt.axhline(y=80, color='#C87867', ls='--', lw=3)
            plt.annotate('Ideal', (0, 90), ha='center', va='center', fontsize=8, color='#C87867')
            plt.axhline(y=100, color='#67BEC8', ls='--', lw=3)
            plt.annotate('Ideal', (0, 110), ha='center', va='center', fontsize=8, color='#67BEC8')
            
               # Language Specific
            if country == 'MX':
                plt.plot(ax1_LnCh_plt, color='#001EBA', label = '% Del Ejercicio', linewidth=2)
                plt.plot(ax2_LnCh_plt, color='#BA0000', label = '% Del Vehículo', linewidth=2)
                plt.title('Resultados de Evasión de Barricada - ' + student)
            else:
                plt.plot(ax1_LnCh_plt, color='#001EBA', label = '% Of The Exercise', linewidth=2)
                plt.plot(ax2_LnCh_plt, color='#BA0000', label = '% Of The Vehicle', linewidth=2)
                plt.title(student + ' ' + 'Barricade Evasion Results')
            
            plt.ylim(ymin=0, ymax=120)
            plt.legend()
            
            #Language specific labels
            if country == 'MX':
                plt.xlabel('Intentos')
            else:
                plt.xlabel('Runs')
            
            plt.ylabel('%')
            plt.tight_layout()
            plt.savefig('plots/LnCh Graph ' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            l_graph = InlineImage(doc, 'plots/LnCh Graph ' + student + '.png')
            
            #Final exercise table population
            # items = mse_report[student]
            
# Final Exercise Percetage Graph - NEW
            column_x = df_final_exercise.loc[(df_final_exercise['studentId'] == studentId)]['final_result']
            row_x = column_x.idxmax()

            # Data Variables
            data_slalom = df_final_exercise.iloc[row_x]['slalom']
            data_lnch = df_final_exercise.iloc[row_x]['LnCh']
            data_rev = df_final_exercise.iloc[row_x]['rev_%']*100
            data_s = df_final_exercise.iloc[row_x]['final_result']*100
            min_std = 80 - data_s
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y = 100 - (min_std + data_s)
            size = [data_s, min_std, data_y]
            marker_a = [79,1,20]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')
            
            # Colors for percetage graph
            if data_s >= 80:
                colors_x = ['lawngreen', 'gainsboro', 'grey']
            elif data_s >= 70 < 80:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_s < 70:
                colors_x = ['darkred', 'gainsboro', 'grey']

            # Overall Perf Graph
            fig, ax = plt.subplots(figsize=(1.5,1.5))
            ax.axis('equal')
            
            ## Local Levels
            if country == 'MX':
                label1 = """Min.80%"""
                label2 = "Desempeño"
            else:
                label1 = """Min.80%"""
                label2 = "Performance"

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, radius = 1.3+.1, colors=['white', 'black', 'white'], labels=pie_3_names, counterclock = False, startangle=-90, textprops={'fontsize': 10})
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 8})
            plt.setp(pie_1, width=0.4, edgecolor='white')
            ax.annotate((str(int(data_s)) + '%'), (0, 0), ha='center', va='center', fontsize=16, fontweight='bold', color='black')
#             ax.annotate((label2), (0, .35), ha='center', va='center', fontsize=8, color='black')
            plt.margins(0,0)
            plt.savefig('plots/final_percent-' + student + '.png', bbox_inches='tight', dpi=300)
#             plt.show()
            plt.close()
            
            pct_graph = InlineImage(doc, 'plots/final_percent-' + student + '.png')
            
#Slalom Gpraph
            min_std = 80 - data_slalom
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y_slalom = 100 - (min_std + data_slalom)
            size = [data_slalom, min_std, data_y_slalom]
            marker_a = [79,1,20]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')

            fig, ax = plt.subplots(figsize=(1,1))
            ax.axis('equal')

            if data_slalom >= 80:
                colors_x = ['lawngreen', 'gainsboro', 'grey']
            elif data_slalom >= 70 < 80:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_slalom < 70:
                colors_x = ['darkred', 'gainsboro', 'grey']

            ## Local Levels
            if country == 'MX':
                label1 = """Min. 80%"""
                label3 = ""
            else:
                label1 = """Min. 80%"""
                label3 = ""

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, 
                                 radius = 1.3+.1, 
                                 colors=['white', 'black', 'white'], 
                                 labels=pie_3_names, 
                                 counterclock = False, 
                                 startangle=-90,
                                 textprops={'fontsize': 8},
                                 rotatelabels = False,
                                 labeldistance = 1
                                )
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 6})
            plt.setp(pie_1, width=0.5, edgecolor='white')
            ax.annotate((str(int(data_slalom)) + '%'), (0, 0), ha='center', va='center', fontsize=10, fontweight='bold', color='black')
            ax.annotate((label3), (0, .35), ha='center', va='center', fontsize=6, color='black')
            plt.margins(0,0)

            plt.savefig('plots/final_slalom_percent-' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close() 
            
            slalom_graph = InlineImage(doc, 'plots/final_slalom_percent-' + student + '.png') #Slalom Graph
            
#LaneChange Gpraph
            min_std = 80 - data_lnch
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y_lnch = 100 - (min_std + data_lnch)
            size = [data_lnch, min_std, data_y_lnch]
            marker_a = [79,1,20]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')

            fig, ax = plt.subplots(figsize=(1,1))
            ax.axis('equal')

            if data_lnch >= 80:
                colors_x = ['lawngreen', 'gainsboro', 'grey']
            elif data_lnch >= 70 < 80:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_lnch < 70:
                colors_x = ['darkred', 'gainsboro', 'grey']

            ## Local Levels
            if country == 'MX':
                label1 = """Min. 80%"""
                label4 = ""
            else:
                label1 = """Min. 80%"""
                label4 = ""

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, 
                                 radius = 1.3+.1, 
                                 colors=['white', 'black', 'white'], 
                                 labels=pie_3_names, 
                                 counterclock = False, 
                                 startangle=-90,
                                 textprops={'fontsize': 8},
                                 rotatelabels = False,
                                 labeldistance = 1
                                )
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 6})
            plt.setp(pie_1, width=0.5, edgecolor='white')
            ax.annotate((str(int(data_lnch)) + '%'), (0, 0), ha='center', va='center', fontsize=10, fontweight='bold', color='black')
            ax.annotate((label4), (0, .35), ha='center', va='center', fontsize=6, color='black')
            plt.margins(0,0)

            plt.savefig('plots/final_lnch_percent-' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            lnch_graph = InlineImage(doc, 'plots/final_lnch_percent-' + student + '.png') #LaneChange Gpraph
            
#Reverse Gpraph
            min_std = 20 - data_rev
            if min_std < 0:
                min_std = 0
            else:
                pass
            data_y_rv = 100 - (min_std + data_rev)
            size = [data_rev, min_std, data_y_rv]
            marker_a = [19,1,80]

            dif_pct = 100 - mseg_perf
            gp_pct = [mseg_perf, dif_pct]
            # min_std = 80
            min_std_df = 100 - min_std
            avg_size = [min_std, min_std_df]
            my_circle = plt.Circle((0,0), 0.7, color='white')

            fig, ax = plt.subplots(figsize=(1,1))
            ax.axis('equal')

            if data_rev > 25:
                colors_x = ['darkred', 'gainsboro', 'grey']
            elif data_rev > 20 <= 25:
                colors_x = ['gold', 'gainsboro', 'grey']
            elif data_rev <= 20:
                colors_x = ['lawngreen', 'gainsboro', 'grey']

            ## Local Levels
            if country == 'MX':
                label1 = """Max. 20%"""
                label5 = ""
            else:
                label1 = """Max. 20%"""
                label5 = ""

            pie_3_names = ['',label1, '']

            pie_back, _ = ax.pie(marker_a, 
                                 radius = 1.3+.1, 
                                 colors=['white', 'black', 'white'], 
                                 labels=pie_3_names, 
                                 counterclock = False, 
                                 startangle=-90, 
                                 textprops={'fontsize': 8},
                                 rotatelabels = False,
                                 labeldistance = 1
                                )
#             print(student, size)
            plt.setp(pie_back, width=0.3, edgecolor='white')
            pie_1, _ = ax.pie(size, radius = 1.3, colors=colors_x, counterclock = False, startangle=-90, textprops={'fontsize': 6})
            plt.setp(pie_1, width=0.5, edgecolor='white')
            ax.annotate((str(int(data_rev)) + '%'), (0, 0), ha='center', va='center', fontsize=10, fontweight='bold', color='black')
            ax.annotate((label5), (0, .35), ha='center', va='center', fontsize=6, color='black')
            plt.margins(0,0)

            plt.savefig('plots/final_rv_percent-' + student + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            
            rev_graph = InlineImage(doc, 'plots/final_rv_percent-' + student + '.png') #Reverse Gpraph
            
# Final Exercise Graph -New


            data_time = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].f_time.dt.total_seconds()
            data_time = data_time.reset_index(drop=True)
            data_performance = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].final_result*100
            data_performance = data_performance.reset_index(drop=True)
            data_rv = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].rev_slalom.dt.total_seconds()
            data_rv = data_rv.reset_index(drop=True)
            
            
            data_tm_fn = data_performance
            cones_fn = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].cones
            cones_fn = cones_fn.reset_index(drop=True)
            gates_fn = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].gates
            gates_fn = gates_fn.reset_index(drop=True)
            xtick = mse_report[student][0]['f_time']
            assets_dir = "../../../static/assets"
            cone_img = mpimg.imread(f'{assets_dir}/cones_for_reports.png')

            av_rev_result = df_final_exercise['rev_slalom'].mean().total_seconds()
            av_gp_time = df_final_exercise['f_time'].mean().total_seconds()

            pass_time = 80 #(ideal_time.total_seconds()*1.2)
            
            # New Values for Graphs #<-----Changed
            data_rv_pct = (data_rv / data_time)*100
            data_per_fn = data_performance - data_rv_pct

            slalom_pct = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].slalom
            slalom_pct = slalom_pct.reset_index(drop=True)
            lnch_pct = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].LnCh
            lnch_pct = lnch_pct.reset_index(drop=True)

            passes = data_time.index.tolist()
            passes.append(2)

            label = df_final_exercise.loc[df_final_exercise['studentId'] == studentId].stress
            if country == 'MX':
                label = label.reset_index(drop=True).replace(0, "Bajo Estrés").replace(1, 'Alto Estrés')
            else:
                label = label.reset_index(drop=True).replace(0, "Low Stress").replace(1, 'High Stress')


            data_sx = mse_report[student][0]['final_result']
            min_stdx = 80 - data_sx
            if min_stdx < 0:
                min_stdx = 0
            else:
                pass
            data_yx = 100 - (min_std + data_s)

            barHeight = .8

            fig, ax = plt.subplots(figsize=(8,4))
            
            ax.invert_yaxis()
            
            imagebox = OffsetImage(cone_img, zoom=0.1)
            imagebox.image.axes = ax

            # Average Bar
            ## labels
            if country == 'MX':
                label3 = """Promedio: """
                label4 = 'Prom. Rev: '
                label5 = """ Prom. Penalización: """
                label6 = 'Prom. Gpo. '
            else:
                label3 = """Average: """
                label4 = 'Av. Rev: '
                label5 = """ Av. Penalties: """
                label6 = 'Gp. Average '
            
            ax.barh('Av', float(mseg_rev), color='khaki', edgecolor='white', height=.3) #<-----Changed
            ax.barh('Av', float(mseg_perf)-float(mseg_rev), left=float(mseg_rev), color='gold', edgecolor='white', height=.3)#<-----Changed
            
            ax.annotate(label3 + str(mseg_t), (mseg_perf, .3), color='black', va='center', ha='right', fontsize=8)
            ax.annotate(label4 + str(mseg_rev), (2, .3), color='black', va='center', ha='left', fontsize=8)
#             ax.annotate(label5 + str(mseg_c) + " | " + str(mseg_g), (mseg_perf, .3), color='black', fontsize=8, va='center', ha='center')
            ax.annotate(label5 + str(mseg_c) + " | " + str(mseg_g), (float(mseg_rev) + 10, .3), color='black', fontsize=8, va='center', ha='left')
            ax.annotate(label6 + str(mseg_perf) + '%', xy=(float(mseg_perf) + .3, 0), xytext=(float(mseg_perf) + 10, 0), ha='left', fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')

            # Main Bar Plot
            ## Labels ##
            if country == 'MX':
                label7 = """T: """ 
                label8 = """Aceptable"""
                label9 = """Penalizaciones: """
            else:
                label7 = """T: """ 
                label8 = """Acceptable"""
                label9 = """Penalties: """

            for i, v in data_tm_fn.items():
            
            # Student Bar
                ax.barh(i+1, data_rv_pct[i], 
                        color='darkred', 
                        edgecolor='white', 
                        height=barHeight) 
                ax.barh(i+1, v - data_rv_pct[i], 
                        left=data_rv_pct[i], 
                        color='red', 
                        edgecolor='white', 
                        height=barHeight) 

                xtick1 = (mse_report[student][i]['f_time'])
                xtick2 = xtick1 #str(datetime.strptime(xtick1, '%M:%S.%f').time()) <------Deprecated (No se necesita con el cambio anterior)
                
                # Time Annotations
                ax.annotate(label7 + xtick2, 
                            (v - 1, i+1), 
                            color='w', fontsize=10, 
                            fontweight='bold', 
                            va='center', 
                            ha='right')
                ax.annotate(str(int(data_performance[i]))+'%', 
                            (v + 1, i+1), 
                            color='black', 
                            fontsize=10, 
                            fontweight='bold', 
                            va='center', 
                            ha='left')

            # Acceptable Reverse Annotation
#                 ax.annotate('', (.1, i + 1.4), xytext=((80 * .2), i + 1.4), #<------ Hay un error en esta formula, no marca la realidad
#                             arrowprops=dict(arrowstyle='|-|'), 
#                             va='top')
#                 ax.annotate(label8,(1, i + 1.5), 
#                             xytext=(80 * .10, i + 1.5), 
#                             va='center', ha='center', fontsize=8)

            # Stress Labels
                if label[i] == "High Stress" or "Alto Estrés":
                    label_color = 'orangered'
                else:
                    label_color = 'yellowgreen'
                
                ax.annotate(label[i], 
                            ((v)+1, i + .8), 
                            ha='left', va='top', 
                            fontsize=8, fontweight='bold', 
                            color=(label_color))
                
                
                # Add Penalties Image
                ab = AnnotationBbox(imagebox, 
                                    ((data_performance[i] - data_rv_pct[i]) - 11, i+1.4),     # Bar Image 
                                    xybox=(0., 0.), 
                                    xycoords='data',
                                    boxcoords="offset points", 
                                    frameon=False)
                ax.add_artist(ab)
                
                # Penalties Cones | Gates
                ax.annotate(label9 + str(cones_fn[i]) + " | " + str(gates_fn[i]), 
                            (data_performance[i] - data_rv_pct[i], i+1.5), 
                            color='black', 
                            fontsize=8, 
                            ha='center', va='center')
                
                # Slalom | LnCh Percetage Annotation
                if data_performance[i] < 60:
                    label10 = 'S' 
                    label11 = 'L'
                    labels_offset = .8
                else:
                    label10 = 'Slalom'
                    label11 = 'LnCh'
                    labels_offset = 1
                ax.annotate(label10 + ' ' + str(slalom_pct[i]) + '%' + " | " + label11 + ' ' + str(lnch_pct[i]) + '%', 
                            (data_rv_pct[i] + 5, i + labels_offset), 
                            color='w', 
                            fontsize = 10, fontweight='bold', 
                            va='center', ha='left')

            
            # Reverse Time Annotation
            for i, v in data_rv.items():
                if country == 'MX':
                    ax.annotate("Reversa:" + str(round(float(v),2)), (2, i+1), color='w', fontsize=10, fontweight='bold', va='center') #Reverse Bar Legend
                else:
                    ax.annotate("Reverse:" + str(round(float(v),2)), (2, i+1), color='w', fontsize=10, fontweight='bold', va='center') #Reverse Bar Legend

            # Pass or Fail Annotation <-----Changed
            ax.axvline(pass_time, linestyle='--', ymax=len(passes), color='aqua')
            if country == 'MX':
                ax.annotate('Necesita Práctica', xy=(pass_time - 23, len(passes)-.3), xytext=(pass_time -1, len(passes)-.3), ha='right', fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')
                ax.annotate('Security Driver', xy=(pass_time + 20, len(passes)-.3), xytext=(pass_time +1, len(passes)-.3), fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')
            else:
                ax.annotate('Needs More Work', xy=(pass_time - 30, len(passes)-.3), xytext=(pass_time -1, len(passes)-.3), ha='right', fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')
                ax.annotate('Security Driver', xy=(pass_time + 20, len(passes)-.3), xytext=(pass_time +1, len(passes)-.3), fontsize=10,
                       arrowprops=dict(arrowstyle='->'), va='center')

            ax.set_yticks([])
            ax.set_xticks([])
            ax.axis('off')

            fig.tight_layout()
            plt.xlim(xmin=0, xmax=100)
            plt.ylim(ymin=-.5, ymax=len(passes))
            plt.savefig('plots/final_exercise-' + student + '.png', bbox_inches='tight', dpi=300)
#             plt.show()
            plt.close()
            
            mse_graph = InlineImage(doc, 'plots/final_exercise-' + student + '.png')
            
            for var in tmplt_context:
                print(var)
                tmplt_context[var] = eval(var)
            
            jinja_env = jinja2.Environment(autoescape=True)
            doc.render(tmplt_context, jinja_env)
            
            doc.save('reports/' + str(student) + ' - Report.docx')
            
            comment = str(paragraph)
            
            i+=1
            
            

In [113]:
for var in tmplt_context:
    print(var)
    print(eval(var))
    # tmplt_context[var] = eval(var)
    

<class 'str'>
student
Juan Guerrero Ortiz
<class 'str'>
company


NameError: name 'company' is not defined

In [92]:
dfs['final_exercise']

,ideal_time sec,cone penalty sec,door penalty sec
0,69,2,5
1,Exercise,Chord,MO
2,Slalom,50,3.625
3,Lane Change,100,13.64


In [115]:
create_report(df_report)

### Reportlab

In [118]:
dfs["course_generals"]

,date,location,program,client,country,units
0,2022-06-30,OFF ROAD Training Center,Habilidades de Manejo Avanzadas [Anti-Secuestro],BASF,MX,KPH


In [129]:
df_report.head(1)

,fullname,studentId,company,program,date,vehicle,s_no_runs,s_passed,prcnt_s_pass,avg_ex_control_s,avg_v_control_s,slalom_max,lc_no_runs,lc_passed,prcnt_lc_pass,avg_ex_control_lc,avg_v_control_lc,lnch_max,comments
0,Juan Guerrero Ortiz,GZJ19570731,Basf,Habilidades de Manejo Avanzadas [Anti-Secuestro],2022-06-30,VW Jetta 2.5 (Capacidad 0.93g),9,6,67.0,97.0,58.0,78.0,6,3,50.0,103.0,57.0,86.0,"Juan tuvo un desempeño correcto, y ha mostrado..."


In [135]:
pwidth

612.0

In [136]:
# Variables
company = dfs["course_generals"].loc[0, "client"]
fulldate = dfs["course_generals"].loc[0, "date"]
vehicle = dfs["car_information"].loc[0, "make"]

page_size = letter
pwidth, pheight = letter
fwidth = pwidth*.8

'-----------------------------------------------------------------------------'
'Register Fonts'
'-----------------------------------------------------------------------------'

font_dir = '../../../static/reports/fonts'
registerFont(TTFont('MontserratBold', f"{font_dir}/Montserrat ExtraBold 800.ttf"))
registerFont(TTFont('MontserratBlack', f'{font_dir}/Montserrat Black 900.ttf'))
registerFont(TTFont('Montserrat', f'{font_dir}/Montserrat-Regular.ttf'))
registerFont(TTFont('MontserratLight', f'{font_dir}/Montserrat Light 300.ttf'))
registerFont(TTFont('MontserratThin', f'{font_dir}/Montserrat Thin 250.ttf'))

from reportlab.lib.fonts import addMapping
# addMapping('SabonRom', 0, 0, 'SabonRom') #normal
# addMapping('SabonRom', 0, 1, 'SabonIta') #italic
addMapping('MontserratLight', 1, 0, 'MontserratBold') #bold
# addMapping('SabonRom', 1, 1, 'SabonBolIta') #italic and bold
# heavy = ParagraphStyle(name='normal', fontName='SabonRom', fontSize=10, leading=1.4*10 )


'-----------------------------------------------------------------------------'
'Template'
'-----------------------------------------------------------------------------'

class MyDocTemplate(BaseDocTemplate):

    def __init__(self, filename, **kw):
        self.allowSplitting = 0
        BaseDocTemplate.__init__(self, filename, **kw)

        template_CoverPage = PageTemplate('CoverPage',
            [Frame(.8*cm, pheight/2-(inch*3), pwidth/2, 230, id='F1', showBoundary=0)], 
            onPage=createCover, 
            )
        
        template_NormalPage = PageTemplate('NormalPage',
                                [Frame(pwidth*.08, 2.5*cm, pwidth*.84, pheight*.8, id='F2', 
                                    showBoundary=0)],
                                onPage=partial(header_and_footer, header_content=header_content, footer_content=footer_content),
                                pagesize=page_size)
        
        template_SecondPage = PageTemplate('SecondPage',
                                [Frame(pwidth*.08, pheight*.37, pwidth*.84, pheight*.50, id='F3', 
                                    showBoundary=0),
                                Frame(pwidth*.08, 3*cm, pwidth*.42, pheight*.25, id='F4', 
                                    showBoundary=0),
                                Frame(pwidth*.50, 3*cm, pwidth*.42, pheight*.25, id='F5', 
                                    showBoundary=0)],
                                onPage=partial(header_and_footer, header_content=header_content, footer_content=footer_content),
                                pagesize=page_size)

        self.addPageTemplates([template_CoverPage,template_NormalPage, template_SecondPage])



    def afterFlowable(self, flowable):
        "Registers TOC entries."
        if flowable.__class__.__name__ == 'Paragraph':
            text = flowable.getPlainText()
            style = flowable.style.name
            if style == 'Heading1':
                self.notify('TOCEntry', (0, text, self.page))
            if style == 'Heading2':
                self.notify('TOCEntry', (1, text, self.page))

h1 = PS(name = 'Heading1',
       fontSize = 18,
       leading = 20,
       fontName='MontserratBold',
       textColor="#C10230",
       spaceAfter = 8,
       spaceBefore = 8)

h2 = PS(name = 'Heading2',
       fontSize = 14,
       fontName='MontserratBlack',
       leading = 16,
       leftIndent = 0)
h3 = PS(name = 'Heading3',
       fontSize = 14,
       fontName='MontserratBold',
       textColor="#C10230",
       leading = 16,
       leftIndent = 10)
subh2 = PS(name = 'SubHeading2',
       fontSize = 14,
       fontName='Montserrat',
       leading = 16,
       leftIndent = 5)
l0 = PS(name = 'list0',
        fontSize = 12,
        leading =14,
        leftIndent=0,
        rightIndent=0,
        spaceBefore = 12,
        spaceAfter =0
        )
pstyle = PS(name='ms',
        fontName='MontserratLight',
        fontSize=10,
        leading =14,
        spaceAfter = 8,
        spaceBefore = 8
        )
no_spacing = PS(name='ms',
        fontName='MontserratLight',
        fontSize=10,
        leading =14,
        spaceAfter = 0,
        spaceBefore = 0
        )
footertxt = PS(name='footer',
        fontName='MontserratLight',
        fontSize=10,
        leading =9,
        leftIndent=40,
        rightIndent=0,
        )
super_h1 = PS(name = 'Super_Heading1',
       fontSize = 80,
       leading = 0,
       fontName='MontserratBold',
       textColor="#C10230",
       spaceAfter = 0,
       spaceBefore = 0)
ttl = PS(name = 'Title',
       fontSize = 24,
       leading = 25,
       fontName='MontserratBlack',
       textColor="#000000",
       spaceAfter = 0,
       spaceBefore = 0)

'-----------------------------------------------------------------------------'
'Table Styles'
'-----------------------------------------------------------------------------'

normalTable = TableStyle([
    ('FONTNAME', (0,0), (0, -1), 'Montserrat'),
    ('FONTNAME', (0,-1), (-1,-1), 'Montserrat'),
    ('FONTNAME', (0,1), (-1,-2), 'MontserratLight'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('ALIGN', (0,0), (-1,0), 'CENTER'),
    ('ALIGN', (1,1), (-1,-1), 'CENTER'),
    ('LINEBELOW', (0,0), (-1,0), 2, colors.black),
    ('LINEABOVE', (0,-1), (-1,-1), .5, colors.black),
    ('LINEBELOW', (0,-1), (-1,-1), 2, colors.black),
])

mseTable = TableStyle([
    ('FONTNAME', (0,0), (0, -1), 'Montserrat'),
    ('FONTNAME', (0,-1), (-1,-1), 'Montserrat'),
    ('FONTNAME', (0,1), (-1,-2), 'MontserratLight'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('ALIGN', (1,0), (-1,0), 'CENTER'),
    ('ALIGN', (1,1), (-1,-1), 'CENTER'),
    ('LINEBELOW', (0,0), (-1,0), 2, colors.black),
    ('LINEABOVE', (0,-1), (-1,-1), .5, colors.black),
    ('LINEBELOW', (0,-1), (-1,-1), 2, colors.black),
    ('BACKGROUND', (0, -1), (-1,-1), colors.lightcoral),
    ('VALIGN', (0, -1), (-1,-1), 'MIDDLE'),
    
])

introTable = TableStyle([
    ('FONTNAME', (0,0), (-1, -1), 'Montserrat'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('ALIGN', (0,0), (-1,-1), 'LEFT'),
    ('LINEBELOW', (0,0), (-1,-1), 1, colors.black),
])

consoleTable = TableStyle([
    ('BOX',(0,0),(-1,-1), 2, colors.firebrick),
    ('ALIGN', (0,0), (0,-1), 'RIGHT'),
    ('ALIGN', (0,1), (-1,-1), 'CENTER'),
    ('VALIGN', (0,1), (-1,-1), 'TOP'),
    ('BACKGROUND', (0, 0), (-1,-1), colors.beige),
#     ('LINEBELOW', (0,0), (-1,-1), 1, colors.black),
])


subTable = TableStyle([
    ('FONTNAME', (0,0), (-1,-1), 'MontserratLight'),
#     ('GRID',(0,0),(-1,-1),.5,colors.black),
    ('VALIGN', (0,0), (-1,-1), 'TOP'),
    ('ALIGN', (0,0), (-1,-1), 'RIGHT'),
    ('TOPPADDING', (0,0), (0,0), 10)
])

'-----------------------------------------------------------------------------'
'Header and Footer'
'-----------------------------------------------------------------------------'
def header(canvas, doc, content):
    canvas.saveState()
    w, h = content.wrap(doc.width, doc.topMargin)
    content.drawOn(canvas, doc.leftMargin-22.4, doc.height + doc.bottomMargin + doc.topMargin - h -10)
    canvas.restoreState()

def footer(canvas, doc, content):
    drawPageNumber(canvas, doc)
    canvas.saveState()
    w, h = content.wrap(doc.width, doc.bottomMargin)
    content.drawOn(canvas, doc.leftMargin-22.4, h)
    canvas.restoreState()

def header_and_footer(canvas, doc, header_content, footer_content):
    header(canvas, doc, header_content)
    footer(canvas, doc, footer_content)

def drawPageNumber(canvas, doc):
    pageNumber = canvas.getPageNumber()
    # canvas.setFont("Helvetica",11)
    # canvas.drawCentredString(17.4*cm, 1.35*cm, 'Page '+str(pageNumber))

def PageNumber(canvas, doc):
    return(canvas.getPageNumber())

header_center_text='header_center_text foo'
footer_center_text='footer_center_text bar' 

'-----------------------------------------------------------------------------'
'Variables and Texts'
'-----------------------------------------------------------------------------'

logo_dir = '../../../static/reports/logos'
logo_int = f'{logo_dir}/Logo---AS3-international-200x200.png'
ceo_cfo = f'{logo_dir}/The-CFO-to-the-CEO---Logo.png'
ceo_cfo_inv = f'{logo_dir}/The-CFO-to-the-CEO---Logo_inverted.png'
as3_logo = f'{logo_dir}/AS3 Driver Training - Logo.png'
chrysler300 = f'{logo_dir}/chrysler_300.png'

text_dir = '../../../static/reports/texts'
if country == 'MX':
    with open(f'{text_dir}/[es]_personal_report_intro.txt', 'r', encoding="utf-8") as file:
        intro_pre = file.read()
        intro_text = intro_pre
    
    with open(f'{text_dir}/[es]slalom_description.txt', 'r', encoding="utf-8") as file:
        slalom_pre = file.read()
        slalom_text = slalom_pre

    with open(f'{text_dir}/[es]final_x_description.txt', 'r', encoding="utf-8") as file:
        mse_pre = file.read()
        mse_text = mse_pre
        
    with open(f'{text_dir}/[es]final_x_intro.txt', 'r', encoding="utf-8") as file:
        mse_pre2 = file.read()
        mse_desc = mse_pre2
    
    with open(f'{text_dir}/[es]final_note.txt', 'r', encoding="utf-8") as file:
        fn_note_pre = file.read()
        fn_note = fn_note_pre
    
    
    mainTitle = 'EVALUACIÓN POST ENTRENAMIENTO (AAR)'
    scndTitle = '<para alignment="center">EVALUACIÓN DE DESEMPEÑO</para>'
    thirdTitle = 'DESEMPEÑO EN EJERCICIOS'
    thirdSubTitle = 'EJERCICIOS DE DESARROLLO DE HABILIDADES'
    fourthSubTitle = 'EJERCICIO FINAL CONBINADO'
    slalomTitle = '<para><b>SLALOM</b> - 4 Conos, Cuerda 13mts</para>'
    LnChTitle = '<para><b>CAMBIO EVASIVO DE CARRIL</b> - Diseño Estándar, Cuerda de 32 mts</para>'
    mse_title = 'EJERCICIO MULTIDISCIPLINARIO – 9 Obstáculos, emboscada fija / en movimiento.'
    mse_consoleTitle = "KPI Personal - Mejor Resultado"
    inst_comments_title = 'COMENTARIOS DEL INSTRUCTOR LIDER'
    stdt_name = '<para><b><i>Nombre del Estudiante</i></b></para>'
    company_label = '<para><b><i>Empresa</i></b></para>'
    pgm_label = '<para><b><i>Programa</i></b></para>'
    pgmDt_label = '<para><b><i>Fecha del Programa</i></b></para>'
    vhcl_label = '<para><b><i>Vehículo</i></b></para>'
    intro_title = 'DESCRIPCIÓN DEL PROGRAMA'
    perf_table_title = 'DESEMPEÑO EN NÚMEROS'
    perf_table_heading = 'Promedio del Grupo'
    perf_table_runs = 'Número de Intentos'
    perf_table_completed = 'Porcentaje de Ejercicios Completados'
    perf_table_percetage = 'Promedio de Porcentaje del Ejercicio'
    perf_table_control = 'Promedio de Control del Vehículo'
    perf_table_maxControl = 'Nivel Máximo de Control Alcanzado'
    mse_barricade = '<para><font fontname="MontserratBold" color="darkred" size="14">Barricada</font></para>'
    mse_slalom = '<para><font fontname="MontserratBold" color="darkred" size="14">Slalom</font></para>'
    mse_reverse = '<para><font fontname="MontserratBold" color="darkred" size="14">Reversa</font></para>'
    mse_ova = '<para><font fontname="MontserratBold" color="darkred" size="14">Desempeño Global</font></para>'
    mse_graphTitle = 'Gráfica de Desempeño'
    mseTable_headers = [['Estrés', 'Tiempo Final', 'Reversa', 'Slalom', 'Barricada', Paragraph('<para align="center">Conos</para>'), 
                         Paragraph('<para align="center">Puertas</para>'), Paragraph('<para align="center">Resultado<br/> Final</para>')]]
    gpavg_lvl = 'Promedio del Grupo'

else:
    with open(f'{text_dir}/[en]_personal_report_intro.txt', 'r', encoding="utf-8") as file:
        intro_pre = file.read()
        intro_text = intro_pre
    
    with open(f'{text_dir}/[en]slalom_description.txt', 'r', encoding="utf-8") as file:
        slalom_pre = file.read()
        slalom_text = slalom_pre

    with open(f'{text_dir}/[en]final_x_description.txt', 'r', encoding="utf-8") as file:
        mse_pre = file.read()
        mse_text = mse_pre
        
    with open(f'{text_dir}/[en]final_x_intro.txt', 'r', encoding="utf-8") as file:
        mse_pre2 = file.read()
        mse_desc = mse_pre2
    
    with open(f'{text_dir}/[en]final_note.txt', 'r', encoding="utf-8") as file:
        fn_note_pre = file.read()
        fn_note = fn_note_pre
    
    mainTitle = 'AFER ACTION REPORT (AAR)'
    scndTitle = '<para alignment="center">PERFORMANCE EVALUATION</para>'
    thirdTitle = 'EXERCISE PERFORMANCE'
    thirdSubTitle = 'SKILL DEVELOPMENT EXERCISES'
    fourthSubTitle = 'COMBINED FINAL EXERCISE'
    slalomTitle = '<para><b>SLALOM</b> - 4 Cones, 50ft Chord</para>'
    LnChTitle = '<para><b>EVASIVE LANE CHANGE</b> - Standard Design, 100ft Chord</para>'
    mse_title = 'MULTI-DISCIPLINARY EXERCISE – 9 Obstacles, fixed / rolling ambush.'
    mse_consoleTitle = "Personal KPI - Top Result"
    inst_comments_title = 'LEAD INSTRUCTOR FEEDBACK'
    stdt_name = '<para><b><i>Student Name</i></b></para>'
    company_label = '<para><b><i>Company</i></b></para>'
    pgm_label = '<para><b><i>Program</i></b></para>'
    pgmDt_label = '<para><b><i>Program Date</i></b></para>'
    vhcl_label = '<para><b><i>Vehicle</i></b></para>'
    intro_title = 'PROGRAM DESCRIPTION'
    perf_table_title = 'PERFORMANCE IN NUMBERS'
    perf_table_heading = 'Group Average'
    perf_table_runs = 'Number of Runs'
    perf_table_completed = 'Percentage of Completed Exercises'
    perf_table_percetage = 'Average of Exercise Percentage'
    perf_table_control = 'Average of Vehicle Control'
    perf_table_maxControl = 'Higest Level of Control Achieved'
    mse_barricade = '<para><font fontname="MontserratBold" color="darkred" size="14">Barricade</font></para>'
    mse_slalom = '<para><font fontname="MontserratBold" color="darkred" size="14">Slalom</font></para>'
    mse_reverse = '<para><font fontname="MontserratBold" color="darkred" size="14">Reverse</font></para>'
    mse_ova = '<para><font fontname="MontserratBold" color="darkred" size="14">Overall Performance</font></para>'
    mse_graphTitle = 'Performance Graph'
    mseTable_headers = [['Stress', 'Final Time', 'Reverse', 'Slalom', 'Barricade', Paragraph('<para align="center">Cone<br/> Penalties</para>'), 
                         Paragraph('<para align="center">Gate<br/> Penalties</para>'), Paragraph('<para align="center">Final<br/> Result</para>')]]
    gpavg_lvl = 'Group Average'
    
'-----------------------------------------------------------------------------'
'Functions'
'-----------------------------------------------------------------------------'

def createCover(canvas, doc):
#     page_width, page_height = canvas._pagesize
    image = pImage.open(logo_int)
    image_width, image_height = image.size
#     draw_width, draw_height = page_width, page_height

#     canvas.drawImage(logo_int,
#                      pwidth/2, 500, #width=draw_width, height=draw_height,
#                      preserveAspectRatio=True)
    canvas.drawImage(logo_int, ((pwidth/2)-(image_width/2)),pheight-image_height*1.5, mask='auto')
    
def create_deco():
    global header_content, footer_content
    '-----------------------------------------------------------------------------'
    'Header'
    '-----------------------------------------------------------------------------'
    Header_caption=((header_center_text))
    A='Author'
    Header_table_data=[[Image(ceo_cfo_inv, width=230, height=73)]
    ]
    
    Header_table=Table(Header_table_data,
                        colWidths=[pwidth*.84],
                        rowHeights=[3*cm],
                        style=[
    # ('GRID',(0,0),(1,2),1,colors.black),
    # #('ALIGN',(0,0),(1,0),'CENTER'),
    ('ALIGN',(0,0),(0,0),'RIGHT'),
    # ('VALIGN',(0,0),(3,2),'MIDDLE'),
    # ('ALIGN',(2,0),(3,2),'RIGHT'),
    # ('SPAN',(0,0),(0,2)),
    # ('SPAN',(1,0),(1,2)),
    # ('BOX',(2,0),(-1,-1),1,colors.black),
    #('TEXTCOLOR',(0,0),(3,0),colors.Color(49/255,71/255,137/255))
    ])
    '-----------------------------------------------------------------------------'
    'Footer'
    '-----------------------------------------------------------------------------'

    Footer_table_data=[[Image(as3_logo, 
                        width=144, 
                        height=60),
                        Paragraph('<para fontname="MontserratBold" size="12" color="black">WWW.</para><para color="#C10230">AS3</para><para color="black">INTERNATIONAL.COM</para>')]
                        ]
    Footer_table=Table(Footer_table_data,colWidths=[(pwidth*.84)/2,(pwidth*.84)/2],rowHeights=[1*cm] ,style=[
    # ('GRID',(0,0),(-1,-1),1,colors.black),
    #('ALIGN',(0,0),(1,0),'CENTER'),
    ('ALIGN',(0,1),(-1,-1),'RIGHT'),
    ('VALIGN',(0,0),(0,0),'MIDDLE'),
    # ('ALIGN',(0,0),(3,0),'CENTER'),
    #('TEXTCOLOR',(0,0),(3,0),colors.Color(49/255,71/255,137/255))
    ])
    
    footer_content = Footer_table
    header_content = Header_table

create_deco() 

'Image Aspect Ratio'

def get_image(path, width=1*cm):
    img = utils.ImageReader(path)
    iw, ih = img.getSize()
    aspect = ih / float(iw)
    return Image(path, width=width, height=(width * aspect))

def altBackground(data, table):
    data_len = len(data)+1

    for each in range(data_len):
        if each % 2 == 0:
            bg_color = colors.white
        else:
            bg_color = colors.lightgrey

        table.setStyle(TableStyle([('BACKGROUND', (0, each), (-1, each), bg_color)]))

'-----------------------------------------------------------------------------'
'Build story'
'-----------------------------------------------------------------------------'

story = []

'-----------------------------------------------------------------------------'
'Intro Page'
'-----------------------------------------------------------------------------'
story.append(Paragraph(mainTitle, ttl))
story.append(Spacer(0,20))
story.append(Paragraph(student, h2))
story.append(Paragraph(company, no_spacing))
story.append(Paragraph(program, no_spacing))
story.append(NextPageTemplate('NormalPage'))
story.append(PageBreak())

'- Page 2 -'
intro_table_data = [[Paragraph(stdt_name), student],
                    [Paragraph(company_label), company],
                    [Paragraph(pgm_label), program],
                    [Paragraph(pgmDt_label), fulldate],
                    [Paragraph(vhcl_label), vehicle]
                   ]

intro_table = Table(intro_table_data, colWidths=[120,200], style=introTable)

story.append(Spacer(0,20))
story.append(Paragraph(scndTitle, ttl))
story.append(Spacer(0,20))
story.append(intro_table)
story.append(Spacer(0,20))
story.append(Paragraph(intro_title, h1))
story.append(Spacer(0,20))
story.append(Paragraph(intro_text, pstyle))
story.append(PageBreak())

'- Page 3 -'
story.append(Paragraph(thirdTitle, h1))
story.append(Paragraph(thirdSubTitle, h2))
story.append(Spacer(0,50))
story.append(Paragraph(slalomTitle, subh2))
story.append(Spacer(0,20))

slalomGraph = Image('plots/Slalom Graph ' + student + '.png', width=pwidth-inch)
# story.append(slalomGraph)
story.append(get_image('plots/Slalom Graph ' + student + '.png', width=pwidth-inch))
story.append(Paragraph(slalom_text, style=footertxt))
story.append(Spacer(0,80))
story.append(Paragraph(perf_table_title, h3))
'------------------------------------------------------------------------------------'
slalom_table_data = [['','',perf_table_heading],
                     [perf_table_completed, snor, gasnor],
                     [perf_table_runs, spoce, gaspoce],
                     [perf_table_percetage, saoep, gasaoep],
                     [perf_table_control, saovc, gasaovc],
                     [perf_table_maxControl, '', str(sfpl)+'%']
                    ]
slalom_table = Table(slalom_table_data, colWidths=[pwidth*.40, pwidth*.20, pwidth*.20], style=normalTable)
altBackground(slalom_table_data, slalom_table)
'------------------------------------------------------------------------------------'
story.append(slalom_table)
story.append(PageBreak())

'- Page 4 -'
story.append(Spacer(0,50))
story.append(Paragraph(LnChTitle, subh2))
story.append(Spacer(0,20))

slalomGraph = Image('plots/LnCh Graph ' + student + '.png', width=pwidth-inch)
# story.append(slalomGraph)
story.append(get_image('plots/LnCh Graph ' + student + '.png', width=pwidth-inch))
story.append(Paragraph(slalom_text, style=footertxt))
story.append(Spacer(0,80))
story.append(Paragraph(perf_table_title, h3))
'------------------------------------------------------------------------------------'
lnch_table_data = [['','',perf_table_heading],
                     [perf_table_completed, lnor, galnor],
                     [perf_table_runs, lpoce, galpoce],
                     [perf_table_percetage, laoep, galaoep],
                     [perf_table_control, laovc, galaovc],
                     [perf_table_maxControl, '', str(lfpl)+'%']
                    ]
lnch_table = Table(lnch_table_data, colWidths=[pwidth*.40, pwidth*.20, pwidth*.20], style=normalTable)
altBackground(lnch_table_data, lnch_table)
'------------------------------------------------------------------------------------'


story.append(lnch_table)
story.append(PageBreak())

'- Page 5 -'

story.append(Paragraph(fourthSubTitle, h1))
story.append(Spacer(0,20))
story.append(Paragraph(mse_desc))
story.append(Spacer(0,20))
story.append(Paragraph(mse_consoleTitle, h3))
story.append(Spacer(0,10))

'------------------------------------------------------------------------------------'
mse_lnch_graph = get_image('plots/final_lnch_percent-' + student + '.png', width=fwidth*.20)
mse_slalom_graph = get_image('plots/final_slalom_percent-' + student + '.png', width=fwidth*.20)
mse_reverse_graph = get_image('plots/final_rv_percent-' + student + '.png', width=fwidth*.20)
mse_ova_graph = get_image('plots/final_percent-' + student + '.png', width=fwidth*.30)


mse_table_data = [[Paragraph(mse_barricade), Paragraph(mse_slalom), Paragraph(mse_reverse), Paragraph(mse_ova)], 
                  [mse_lnch_graph, mse_slalom_graph, mse_reverse_graph, mse_ova_graph]
                 ]
mse_table = Table(mse_table_data, colWidths=[fwidth*.23, fwidth*.23, fwidth*.23, fwidth*.34], style=consoleTable)
'------------------------------------------------------------------------------------'

story.append(mse_table)
story.append(Spacer(0,5))
story.append(Paragraph(mse_text))
story.append(PageBreak())

'- Page 6 -'
msei = get_image('plots/final_exercise-' + student + '.png', width=fwidth)
msei_data = [[msei]]

# story.append(get_image('plots/final_exercise-' + student + '.png', width=fwidth))
story.append(Spacer(0,20))

'------------------------------------------------------------------------------------'
mse_hist_last_line = [[Paragraph(gpavg_lvl), mseg_t, str(mseg_rev_pc)+'%', '','', mseg_c, mseg_g, str(mseg_perf)+'%']]
mse_hist_data = mseTable_headers + np.array(mse_data.values).tolist() + np.array(mse_hist_last_line).tolist()

mse_table = Table(mse_hist_data, colWidths=[pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1, pwidth*.1], style=mseTable)

story.append(Paragraph(mse_graphTitle, h3))
story.append(Spacer(0,10))
story.append(mse_table)
story.append(Spacer(0,20))
story.append(Table(msei_data, colWidths=None, rowHeights=None, style=consoleTable))
story.append(Spacer(0,20))
story.append(Paragraph(inst_comments_title, h3))
story.append(Spacer(0,20))
story.append(Paragraph(comment))
story.append(Spacer(0,20))
story.append(Paragraph(fn_note))

'----------------------------------------------------------------'
'Document Creation'
'----------------------------------------------------------------'

doc = MyDocTemplate('AS3_Student_Report '+ company + '_Sample.pdf', pagesize=letter)
doc.multiBuild(story)

NameError: name 'sfpl' is not defined

In [280]:
doc

In [179]:
open('../../../static/reports/texts/[es]_personal_report_intro.txt', "r")

FileNotFoundError: [Errno 2] No such file or directory: '../../../static/reports/texts/[es]_personal_report_intro.txt'